In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 0: 설정
# ══════════════════════════════════════════════════════════════

"""
채권 금리 기술적 분석 시스템 — 통합 설정

모든 사용자 설정을 이 셀에서 관리합니다.
- 데이터 소스 / 분석 대상 / 기간
- 상관분석 파라미터
- 대시보드 오버레이 / 서브패널
- 시그널 게이지 기간
- 저장 경로 / 토글
"""

from pathlib import Path
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


# ============================================================================
# 1. 사용자 실행 설정 (★ 여기만 수정하세요 ★)
# ============================================================================

@dataclass
class ExecutionConfig:
    """실행 시 사용자가 변경하는 설정"""

    # ── 공통 ──
    data_source: str = 'bond_ohlcv'
    # 'bond_close' : Rates시트 종가만 (Tier 2)
    # 'bond_ohlcv' : 국고지표정보 수익률 OHLCV (Tier 1b)
    # 'futures_price' : 선물 가격 OHLCV (Tier 1a)
    # 'futures_yield' : 선물 내재수익률 (Tier 1a, yield만)

    target: str = '국고 3Y'
    start_date: Optional[str] = None   # 'YYYY-MM-DD' 또는 None(전체)
    end_date: Optional[str] = None

    # ── 저장 ──
    save_dir: Path = Path('output')
    save_excel: bool = True
    save_graphs: bool = True

    # ── 상관분석 (Cell 3) ──
    corr_columns: List[str] = field(default_factory=lambda: [
        '국고채RF_3Y', '산금채AAA_5Y', '카드채AA+_3Y',
    ])
    corr_mode: str = 'spreads_changes'
    # 'rates', 'rates_changes', 'spreads', 'spreads_changes',
    # 'spreads_vs_ktb', 'spreads_vs_ktb_changes'

    compare_columns: List[str] = field(default_factory=lambda: [
        '국고채RF_3Y', '국고채RF_5Y', '국고채RF_10Y',
        '카드채AA+_1Y', '카드채AA+_2Y', '카드채AA+_3Y',
    ])
    compare_start: Optional[str] = '2024-01-01'
    compare_end: Optional[str] = None

    rolling_window: int = 20   # 롤링 상관계수 윈도우

    # ── 대시보드 (Cell 4) ──
    show_special_charts: bool = True

    # ── 시그널 (Cell 5) ──
    gauge_short: int = 20     # 약 1개월
    gauge_medium: int = 120   # 약 6개월
    gauge_long: int = 250     # 약 1년


# 전역 실행 설정 인스턴스
CFG = ExecutionConfig()


# ============================================================================
# 2. 파일 및 데이터 설정
# ============================================================================

@dataclass
class DataConfig:
    """데이터 소스 관련 설정"""
    file_path: Path = Path("TA_rawdata.xlsb")
    rates_sheet: str = "Rates"
    spread_sheet: str = "Spread"
    header_row: int = 4
    data_start_row: int = 5
    date_column: str = "A"
    date_format: str = "%Y-%m-%d"


# ============================================================================
# 3. 컬럼명 파싱 규칙
# ============================================================================

@dataclass
class ColumnParsingConfig:
    known_sectors: Tuple[str, ...] = (
        "국고채", "통안채", "공사채", "은행채", "회사채",
        "카드채", "캐피탈", "여전채", "지방채", "특수채",
        "산금채", "수출입", "한전채", "도로공", "주금공",
        "CD", "CP", "콜", "REPO"
    )
    known_ratings: Tuple[str, ...] = (
        "AAA", "AA+", "AA0", "AA-", "AA",
        "A+", "A0", "A-", "A",
        "BBB+", "BBB0", "BBB-", "BBB",
        "BB+", "BB0", "BB-", "BB",
        "B+", "B0", "B-", "B",
        "RF", "무등급"
    )
    maturity_pattern: str = r"(\d+)(D|W|M|Y)"
    maturity_to_years: Dict[str, float] = field(default_factory=lambda: {
        "D": 1/365, "W": 1/52, "M": 1/12, "Y": 1.0
    })


# ============================================================================
# 4. 분석 파라미터
# ============================================================================

@dataclass
class AnalysisConfig:
    rolling_windows: Dict[str, int] = field(default_factory=lambda: {
        "short": 20, "medium": 60, "long": 120
    })
    correlation_threshold: float = 0.7
    min_observations: int = 30
    coint_significance: float = 0.05
    adf_max_lags: int = 12
    zscore_entry_threshold: float = 2.0
    zscore_exit_threshold: float = 0.5
    lookback_period: int = 60
    regime_window: int = 60
    volatility_percentile: float = 0.75
    max_lag: int = 20


# ============================================================================
# 5. 시각화 설정
# ============================================================================

@dataclass
class VisualizationConfig:
    figure_dpi: int = 100
    figure_facecolor: str = "white"
    figsize_single: Tuple[float, float] = (10, 6)
    figsize_double: Tuple[float, float] = (12, 8)
    figsize_multi: Tuple[float, float] = (14, 10)
    figsize_heatmap: Tuple[float, float] = (12, 10)
    colors: Dict[str, str] = field(default_factory=lambda: {
        "primary": "#1f77b4", "secondary": "#ff7f0e",
        "positive": "#2ca02c", "negative": "#d62728",
        "neutral": "#7f7f7f", "highlight": "#9467bd",
        "background": "#f0f0f0", "grid": "#d0d0d0",
    })
    line_colors: Tuple[str, ...] = (
        "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
        "#9467bd", "#8c564b", "#e377c2", "#7f7f7f"
    )
    font_family: str = "Malgun Gothic"
    font_size_title: int = 14
    font_size_label: int = 11
    font_size_tick: int = 9
    font_size_legend: int = 9
    font_size_annotation: int = 8
    linewidth_main: float = 1.5
    linewidth_secondary: float = 1.0
    linewidth_reference: float = 0.8
    linestyle_reference: str = "--"
    grid_alpha: float = 0.3
    grid_linestyle: str = "-"
    grid_linewidth: float = 0.5
    legend_loc: str = "best"
    legend_frameon: bool = True
    legend_framealpha: float = 0.9
    tight_layout_pad: float = 2.0
    subplot_hspace: float = 0.3
    subplot_wspace: float = 0.3


def apply_professional_style():
    viz = VisualizationConfig()
    plt.rcParams.update({
        "font.family": viz.font_family,
        "font.size": viz.font_size_tick,
        "axes.titlesize": viz.font_size_title,
        "axes.labelsize": viz.font_size_label,
        "xtick.labelsize": viz.font_size_tick,
        "ytick.labelsize": viz.font_size_tick,
        "legend.fontsize": viz.font_size_legend,
        "figure.dpi": viz.figure_dpi,
        "figure.facecolor": viz.figure_facecolor,
        "figure.autolayout": False,
        "axes.facecolor": "white",
        "axes.edgecolor": "black",
        "axes.linewidth": 0.8,
        "axes.grid": True,
        "axes.axisbelow": True,
        "grid.alpha": viz.grid_alpha,
        "grid.linestyle": viz.grid_linestyle,
        "grid.linewidth": viz.grid_linewidth,
        "grid.color": viz.colors["grid"],
        "lines.linewidth": viz.linewidth_main,
        "lines.markersize": 4,
        "legend.frameon": viz.legend_frameon,
        "legend.framealpha": viz.legend_framealpha,
        "legend.edgecolor": "black",
        "xtick.direction": "out",
        "ytick.direction": "out",
        "xtick.major.size": 4,
        "ytick.major.size": 4,
        "axes.unicode_minus": False,
    })


# ============================================================================
# 6. 기술적 지표 설정
# ============================================================================

@dataclass
class IndicatorConfig:
    sma_periods: List[int] = field(default_factory=lambda: [5, 10, 20, 60, 120])
    ema_periods: List[int] = field(default_factory=lambda: [12, 26])
    bb_period: int = 20
    bb_std: float = 2.0
    macd_fast: int = 12
    macd_slow: int = 26
    macd_signal: int = 9
    rsi_period: int = 14
    stoch_k: int = 14
    stoch_d: int = 3
    stoch_smooth: int = 3
    williams_period: int = 14
    cci_period: int = 20
    dmi_period: int = 14
    atr_period: int = 14
    chaikin_ema_period: int = 10
    chaikin_roc_period: int = 10
    ichimoku_tenkan: int = 9
    ichimoku_kijun: int = 26
    ichimoku_senkou_b: int = 52
    sar_af_init: float = 0.02
    sar_af_step: float = 0.02
    sar_af_max: float = 0.20
    dema_period: int = 20
    t3_period: int = 5
    t3_vfactor: float = 0.7
    vidya_period: int = 20
    sonar_period: int = 14
    cmf_period: int = 21
    mcclellan_short: int = 19
    mcclellan_long: int = 39
    three_line_break_lines: int = 3
    pnf_box_size: float = 0.01
    pnf_reversal: int = 3
    rsi_overbought: float = 70.0
    rsi_oversold: float = 30.0
    stoch_overbought: float = 80.0
    stoch_oversold: float = 20.0
    cci_overbought: float = 100.0
    cci_oversold: float = -100.0
    williams_overbought: float = -20.0
    williams_oversold: float = -80.0
    gauge_short: int = 20
    gauge_medium: int = 120
    gauge_long: int = 250


# ============================================================================
# 7. OHLCV / 선물 설정
# ============================================================================

@dataclass
class OHLCVConfig:
    sheet_name: str = "국고지표정보"
    instruments: Dict[str, int] = field(default_factory=lambda: {
        "국고 2Y": 1, "국고 3Y": 27, "국고 5Y": 52, "국고 10Y": 77,
        "국고 20Y": 102, "국고 30Y": 127, "국고 50Y": 152,
        "통안 2Y": 177, "통안 3Y": 202,
    })
    date_col_offset: int = 0
    block_size_first: int = 26
    block_size_rest: int = 25
    col_map_first: Dict[str, int] = field(default_factory=lambda: {
        "open": 7, "high": 8, "low": 9, "close": 10,
        "close_eval": 1, "price": 2, "duration": 3, "net_volume": 11,
    })
    col_map_rest: Dict[str, int] = field(default_factory=lambda: {
        "open": 6, "high": 7, "low": 8, "close": 9,
        "close_eval": 0, "price": 1, "duration": 2, "net_volume": 10,
    })
    header_row: int = 3
    data_start_row: int = 4


@dataclass
class FuturesConfig:
    sheet_name: str = "Futures"
    instruments: Dict[str, int] = field(default_factory=lambda: {
        "3년국채 연결": 1, "5년국채 연결": 51,
        "10년국채 연결": 100, "30년국채 연결": 149,
    })
    block_size_first: int = 51
    block_size_rest: int = 50
    col_map_first: Dict[str, int] = field(default_factory=lambda: {
        "close": 1, "open": 2, "high": 3, "low": 4, "volume": 5,
        "yield": 6, "settle": 7, "volatility": 10, "oi": 21, "duration": 18,
    })
    col_map_rest: Dict[str, int] = field(default_factory=lambda: {
        "close": 0, "open": 1, "high": 2, "low": 3, "volume": 4,
        "yield": 5, "settle": 6, "volatility": 9, "oi": 20, "duration": 17,
    })
    header_row: int = 3
    data_start_row: int = 4


# ============================================================================
# 8. 전역 설정 인스턴스
# ============================================================================

DATA_CONFIG = DataConfig()
COLUMN_CONFIG = ColumnParsingConfig()
ANALYSIS_CONFIG = AnalysisConfig()
VIZ_CONFIG = VisualizationConfig()
INDICATOR_CONFIG = IndicatorConfig()
OHLCV_CONFIG = OHLCVConfig()
FUTURES_CONFIG = FuturesConfig()


# ============================================================================
# 9. 공통 차트 유틸리티
# ============================================================================

A4_PORTRAIT = (8.27, 11.69)    # inches (210mm x 297mm)
A4_LANDSCAPE = (11.69, 8.27)   # inches (297mm x 210mm)

CHART_STYLE = {
    'linewidth': 0.75,
    'grid_alpha': 0.3,
    'grid_linewidth': 0.5,
    'title_fontsize': 9,
    'label_fontsize': 7,
    'tick_fontsize': 6,
    'legend_fontsize': 6,
    'color_primary': 'black',
    'color_secondary': '#1a5276',
}


def apply_chart_style(ax):
    """공통 차트 스타일 적용"""
    ax.grid(True, alpha=CHART_STYLE['grid_alpha'], linewidth=CHART_STYLE['grid_linewidth'])
    ax.tick_params(axis='both', labelsize=CHART_STYLE['tick_fontsize'])
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)


def setup_date_axis(ax, dates):
    """기간 길이에 따른 자동 날짜 포맷"""
    if len(dates) < 2:
        return
    span_days = (dates[-1] - dates[0]).days
    if span_days <= 180:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%y.%m.%d'))
        ax.xaxis.set_major_locator(mdates.MonthLocator())
    elif span_days <= 730:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%y.%m'))
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
    else:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%y.%m'))
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    ax.tick_params(axis='x', rotation=30, labelsize=CHART_STYLE['tick_fontsize'])


# 스타일 적용
apply_professional_style()

print("Cell 0 완료: 설정 로딩")


In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 1: 모듈 로딩 (유틸리티 + 엔진 + 렌더러 + 시그널)
# ══════════════════════════════════════════════════════════════

"""
통합 모듈: 데이터 로더, 전처리, 상관분석, 공적분, 시각화,
         기술적 지표 엔진, 차트 렌더러, 시그널 분류
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
import re
import warnings
import logging
from pathlib import Path
from datetime import datetime, timedelta
from typing import Optional, Dict, List, Tuple, Any, Union, Set
from dataclasses import dataclass, field
from functools import cached_property
from enum import Enum
from scipy import stats
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

try:
    from statsmodels.tsa.stattools import adfuller, coint
    from statsmodels.regression.linear_model import OLS
    from statsmodels.tools import add_constant
    HAS_STATSMODELS = True
except ImportError:
    HAS_STATSMODELS = False

logger = logging.getLogger(__name__)


# ============================================================================
# Part 1: pyxlsb 데이터 로더 유틸리티
# ============================================================================

def excel_serial_to_datetime(serial):
    """Excel 날짜 시리얼 번호를 datetime으로 변환"""
    if serial is None:
        return None
    if isinstance(serial, str):
        return None
    try:
        serial = float(serial)
        if serial < 1 or serial > 100000:
            return None
        base = datetime(1899, 12, 30)
        return base + timedelta(days=serial)
    except (ValueError, TypeError, OverflowError):
        return None


def read_xlsb_raw(file_path, sheet_name):
    """pyxlsb로 xlsb 시트의 전체 데이터를 행 리스트로 읽기"""
    from pyxlsb import open_workbook
    rows = []
    with open_workbook(str(file_path)) as wb:
        with wb.get_sheet(sheet_name) as ws:
            for row in ws.rows():
                rows.append(tuple(c.v for c in row))
    return rows


def read_rates_dataframe(file_path, sheet_name, header_row=4, data_start_row=5):
    """금리/스프레드 시트를 DataFrame으로 읽기"""
    raw_rows = read_xlsb_raw(file_path, sheet_name)
    if len(raw_rows) < data_start_row:
        return pd.DataFrame()

    header_idx = header_row - 1
    headers = []
    for i, v in enumerate(raw_rows[header_idx]):
        if v is not None and str(v).strip():
            headers.append(str(v).strip())
        else:
            headers.append(f'_col_{i}')

    data_idx = data_start_row - 1
    dates = []
    data_rows = []

    for row in raw_rows[data_idx:]:
        if not row or row[0] is None:
            continue
        dt = excel_serial_to_datetime(row[0])
        if dt is None:
            continue
        dates.append(pd.Timestamp(dt))
        vals = []
        for v in row[1:len(headers)]:
            try:
                vals.append(float(v) if v is not None else np.nan)
            except (ValueError, TypeError):
                vals.append(np.nan)
        while len(vals) < len(headers) - 1:
            vals.append(np.nan)
        data_rows.append(vals)

    if not dates:
        return pd.DataFrame()

    df = pd.DataFrame(data_rows, index=pd.DatetimeIndex(dates, name='Date'),
                      columns=headers[1:len(headers)])
    df = df.sort_index()
    df = df.dropna(how='all')
    return df


def convert_date_column(df, date_col):
    """DataFrame의 날짜 컬럼 변환"""
    df = df.copy()
    if date_col not in df.columns:
        raise ValueError(f"날짜 컬럼 '{date_col}'을 찾을 수 없습니다.")
    df[date_col] = df[date_col].apply(
        lambda v: pd.Timestamp(excel_serial_to_datetime(v)) if not isinstance(v, (pd.Timestamp, datetime)) else v
    )
    return df


# ============================================================================
# Part 2: 데이터 전처리 및 컬럼 파싱
# ============================================================================

@dataclass
class BondInfo:
    """채권 정보 데이터 클래스"""
    raw_name: str
    sector: Optional[str] = None
    rating: Optional[str] = None
    maturity: Optional[str] = None
    maturity_years: Optional[float] = None
    is_bond: bool = False

    def __str__(self):
        if self.is_bond:
            return f"{self.sector}({self.rating}) {self.maturity}"
        return self.raw_name

    def to_dict(self):
        return {
            "raw_name": self.raw_name, "sector": self.sector,
            "rating": self.rating, "maturity": self.maturity,
            "maturity_years": self.maturity_years, "is_bond": self.is_bond
        }


class ColumnParser:
    """채권 컬럼명 파서"""

    def __init__(self, config=None):
        self.config = config or COLUMN_CONFIG
        self._maturity_regex = re.compile(self.config.maturity_pattern)
        self._sorted_sectors = sorted(self.config.known_sectors, key=len, reverse=True)
        self._sorted_ratings = sorted(self.config.known_ratings, key=len, reverse=True)

    def parse(self, column_name):
        info = BondInfo(raw_name=column_name)
        name = column_name.strip()

        sector = None
        remaining = name
        for s in self._sorted_sectors:
            if name.startswith(s):
                sector = s
                remaining = name[len(s):]
                break
        if sector is None:
            return info
        info.sector = sector

        if "_" in remaining:
            parts = remaining.split("_", 1)
            rating_part, maturity_part = parts[0], parts[1]
        else:
            match = re.search(r'\d', remaining)
            if match:
                rating_part = remaining[:match.start()]
                maturity_part = remaining[match.start():]
            else:
                rating_part, maturity_part = remaining, ""

        rating = None
        for r in self._sorted_ratings:
            if rating_part.upper() == r.upper() or rating_part.startswith(r):
                rating = r
                break
        if rating is None:
            rating = rating_part if rating_part else "무등급"
        info.rating = rating

        maturity_match = self._maturity_regex.search(maturity_part)
        if maturity_match:
            num = int(maturity_match.group(1))
            unit = maturity_match.group(2).upper()
            info.maturity = f"{num}{unit}"
            info.maturity_years = num * self.config.maturity_to_years.get(unit, 1.0)
            info.is_bond = True
        else:
            info.maturity = maturity_part if maturity_part else None
        return info

    def parse_all(self, columns):
        return {col: self.parse(col) for col in columns}

    def parse_to_dataframe(self, columns):
        parsed = self.parse_all(columns)
        return pd.DataFrame([info.to_dict() for info in parsed.values()])


class BondDataPreprocessor:
    """채권 데이터 전처리기"""

    def __init__(self, data):
        self._original_data = data.copy()
        self._data = data.copy()
        self._parser = ColumnParser()
        self._column_info = self._parser.parse_all(data.columns.tolist())
        self._info_df = self._parser.parse_to_dataframe(data.columns.tolist())

    @property
    def data(self):
        return self._data.copy()

    @property
    def columns(self):
        return self._data.columns.tolist()

    @property
    def column_info(self):
        return self._info_df.copy()

    @cached_property
    def sectors(self):
        return sorted(self._info_df[self._info_df["is_bond"]]["sector"].unique())

    @cached_property
    def ratings(self):
        return sorted(self._info_df[self._info_df["is_bond"]]["rating"].unique())

    @cached_property
    def maturities(self):
        df = self._info_df[self._info_df["is_bond"]].copy()
        df = df.sort_values("maturity_years")
        return df["maturity"].unique().tolist()

    def reset(self):
        self._data = self._original_data.copy()
        return self

    def filter_bonds_only(self):
        bond_cols = [col for col, info in self._column_info.items() if info.is_bond]
        self._data = self._data[bond_cols]
        return self

    def filter_sector(self, sectors):
        if isinstance(sectors, str):
            sectors = [sectors]
        cols = [col for col, info in self._column_info.items()
                if info.sector in sectors and col in self._data.columns]
        self._data = self._data[cols]
        return self

    def filter_rating(self, ratings):
        if isinstance(ratings, str):
            ratings = [ratings]
        cols = [col for col, info in self._column_info.items()
                if info.rating in ratings and col in self._data.columns]
        self._data = self._data[cols]
        return self

    def filter_maturity(self, maturities=None, min_years=None, max_years=None):
        if maturities is not None:
            if isinstance(maturities, str):
                maturities = [maturities]
            cols = [col for col, info in self._column_info.items()
                    if info.maturity in maturities and col in self._data.columns]
        else:
            cols = []
            for col, info in self._column_info.items():
                if col not in self._data.columns or not info.is_bond:
                    continue
                if info.maturity_years is None:
                    continue
                if min_years is not None and info.maturity_years < min_years:
                    continue
                if max_years is not None and info.maturity_years > max_years:
                    continue
                cols.append(col)
        self._data = self._data[cols]
        return self

    def filter_date_range(self, start=None, end=None):
        if start is not None:
            self._data = self._data[self._data.index >= str(start)[:10]]
        if end is not None:
            self._data = self._data[self._data.index <= str(end)[:10]]
        return self

    def select_columns(self, columns):
        valid_cols = [c for c in columns if c in self._data.columns]
        self._data = self._data[valid_cols]
        return self

    def dropna(self, how="any", thresh=None):
        if thresh is not None:
            self._data = self._data.dropna(thresh=thresh)
        else:
            self._data = self._data.dropna(how=how)
        return self

    def fillna(self, method="ffill", limit=None):
        if method == "interpolate":
            self._data = self._data.interpolate(method="linear", limit=limit)
        elif method == "ffill":
            self._data = self._data.ffill(limit=limit)
        elif method == "bfill":
            self._data = self._data.bfill(limit=limit)
        else:
            self._data = self._data.fillna(value=method)
        return self

    def get_data(self):
        return self._data.copy()

    def get_yield_curve(self, sector, rating=None, date=None):
        prep = BondDataPreprocessor(self._original_data)
        prep.filter_sector(sector)
        if rating:
            prep.filter_rating(rating)
        data = prep.get_data()
        if date is None:
            date = data.index.max()
        row = data.loc[date]
        result = {}
        for col in row.index:
            info = self._column_info.get(col)
            if info and info.maturity_years is not None:
                result[info.maturity_years] = row[col]
        curve = pd.Series(result).sort_index()
        curve.name = f"{sector} Yield Curve ({date})"
        return curve

    def get_spread(self, col1, col2):
        spread = self._original_data[col1] - self._original_data[col2]
        spread.name = f"{col1} - {col2}"
        return spread

    def get_summary(self):
        return self._data.describe()


def create_spread_matrix(data, base_column):
    """기준 금리 대비 스프레드 매트릭스 생성"""
    spread_df = data.sub(data[base_column], axis=0)
    spread_df = spread_df.drop(columns=[base_column])
    return spread_df


def calculate_changes(data, periods=1):
    """금리 변화 계산 (bp 단위)"""
    return data.diff(periods) * 100


# ============================================================================
# Part 3: 상관관계 분석
# ============================================================================

@dataclass
class CorrelationResult:
    correlation_matrix: pd.DataFrame
    pvalue_matrix: pd.DataFrame
    method: str
    n_observations: int
    start_date: pd.Timestamp
    end_date: pd.Timestamp

    def get_significant_pairs(self, threshold=0.7, pvalue_threshold=0.05):
        pairs = []
        cols = self.correlation_matrix.columns
        n = len(cols)
        for i in range(n):
            for j in range(i + 1, n):
                corr = self.correlation_matrix.iloc[i, j]
                pval = self.pvalue_matrix.iloc[i, j]
                if abs(corr) >= threshold and pval <= pvalue_threshold:
                    pairs.append({"pair_1": cols[i], "pair_2": cols[j],
                                  "correlation": corr, "p_value": pval})
        result = pd.DataFrame(pairs)
        if not result.empty:
            result = result.sort_values("correlation", key=abs, ascending=False)
        return result


@dataclass
class CrossCorrelationResult:
    series1_name: str
    series2_name: str
    correlations: pd.Series
    optimal_lag: int
    max_correlation: float

    def interpret(self):
        if self.optimal_lag > 0:
            return f"'{self.series1_name}'이 '{self.series2_name}'를 {self.optimal_lag}일 선행"
        elif self.optimal_lag < 0:
            return f"'{self.series1_name}'이 '{self.series2_name}'를 {abs(self.optimal_lag)}일 후행"
        return "동시 움직임"


class StaticCorrelationAnalyzer:
    def __init__(self, data):
        self.data = data.copy()

    def calculate(self, method="pearson", min_periods=None):
        if min_periods is None:
            min_periods = ANALYSIS_CONFIG.min_observations
        corr_matrix = self.data.corr(method=method, min_periods=min_periods)
        pval_matrix = self._calculate_pvalues(method)
        return CorrelationResult(
            correlation_matrix=corr_matrix, pvalue_matrix=pval_matrix,
            method=method, n_observations=len(self.data),
            start_date=self.data.index.min(), end_date=self.data.index.max()
        )

    def _calculate_pvalues(self, method):
        cols = self.data.columns
        n = len(cols)
        pvals = np.ones((n, n))
        for i in range(n):
            for j in range(i + 1, n):
                mask = self.data[[cols[i], cols[j]]].notna().all(axis=1)
                x, y = self.data.loc[mask, cols[i]], self.data.loc[mask, cols[j]]
                if len(x) < 3:
                    continue
                if method == "pearson":
                    _, pval = stats.pearsonr(x, y)
                else:
                    _, pval = stats.spearmanr(x, y)
                pvals[i, j] = pvals[j, i] = pval
        return pd.DataFrame(pvals, index=cols, columns=cols)

    def find_most_correlated(self, column, n=5, method="pearson"):
        result = self.calculate(method)
        corrs = result.correlation_matrix[column].drop(column)
        top_n = corrs.abs().nlargest(n).index
        return pd.DataFrame({
            "correlation": corrs[top_n],
            "abs_correlation": corrs[top_n].abs()
        }).sort_values("abs_correlation", ascending=False)


class RollingCorrelationAnalyzer:
    def __init__(self, data):
        self.data = data.copy()

    def calculate_pairwise(self, col1, col2, window=None, min_periods=None):
        if window is None:
            window = ANALYSIS_CONFIG.rolling_windows["medium"]
        if min_periods is None:
            min_periods = window // 2
        rolling_corr = self.data[col1].rolling(window=window, min_periods=min_periods).corr(self.data[col2])
        rolling_corr.name = f"Corr({col1}, {col2})"
        return rolling_corr

    def calculate_multi_window(self, col1, col2):
        result = pd.DataFrame(index=self.data.index)
        for name, window in ANALYSIS_CONFIG.rolling_windows.items():
            result[f"{name}_{window}d"] = self.calculate_pairwise(col1, col2, window)
        return result

    def find_regime_changes(self, col1, col2, window=None, threshold=0.3):
        corr = self.calculate_pairwise(col1, col2, window)
        daily_change = corr.diff()
        weekly_change = corr.diff(5)
        regime_changes = []
        for date in corr.index:
            d_change = abs(daily_change.get(date, 0))
            w_change = abs(weekly_change.get(date, 0))
            if d_change >= threshold or w_change >= threshold * 1.5:
                regime_changes.append({
                    "date": date, "correlation": corr[date],
                    "daily_change": daily_change.get(date, np.nan),
                    "weekly_change": weekly_change.get(date, np.nan)
                })
        return pd.DataFrame(regime_changes)


class CrossCorrelationAnalyzer:
    def __init__(self, data):
        self.data = data.copy()

    def calculate(self, col1, col2, max_lag=None):
        if max_lag is None:
            max_lag = ANALYSIS_CONFIG.max_lag
        s1 = self.data[col1].dropna()
        s2 = self.data[col2].dropna()
        common_idx = s1.index.intersection(s2.index)
        s1, s2 = s1.loc[common_idx], s2.loc[common_idx]
        correlations = {}
        for lag in range(-max_lag, max_lag + 1):
            if lag > 0:
                x, y = s1.iloc[:-lag], s2.iloc[lag:]
            elif lag < 0:
                x, y = s1.iloc[-lag:], s2.iloc[:lag]
            else:
                x, y = s1, s2
            if len(x) > 2:
                corr, _ = stats.pearsonr(x, y)
                correlations[lag] = corr
            else:
                correlations[lag] = np.nan
        corr_series = pd.Series(correlations)
        optimal_lag = corr_series.abs().idxmax()
        return CrossCorrelationResult(
            series1_name=col1, series2_name=col2,
            correlations=corr_series, optimal_lag=optimal_lag,
            max_correlation=corr_series[optimal_lag]
        )

    def analyze_lead_lag_matrix(self, columns=None, max_lag=None):
        if columns is None:
            columns = self.data.columns.tolist()
        if max_lag is None:
            max_lag = ANALYSIS_CONFIG.max_lag
        n = len(columns)
        lag_matrix = pd.DataFrame(np.zeros((n, n)), index=columns, columns=columns)
        for i, col1 in enumerate(columns):
            for j, col2 in enumerate(columns):
                if i >= j:
                    continue
                result = self.calculate(col1, col2, max_lag)
                lag_matrix.iloc[i, j] = result.optimal_lag
                lag_matrix.iloc[j, i] = -result.optimal_lag
        return lag_matrix


class CorrelationClusterer:
    def __init__(self, correlation_matrix):
        self.corr_matrix = correlation_matrix.copy()
        self.distance_matrix = 1 - self.corr_matrix.abs()

    def hierarchical_clustering(self, method="ward", n_clusters=None, distance_threshold=None):
        dist_array = squareform(self.distance_matrix.values)
        linkage_matrix = hierarchy.linkage(dist_array, method=method)
        if n_clusters is not None:
            labels = hierarchy.fcluster(linkage_matrix, n_clusters, criterion="maxclust")
        elif distance_threshold is not None:
            labels = hierarchy.fcluster(linkage_matrix, distance_threshold, criterion="distance")
        else:
            labels = hierarchy.fcluster(linkage_matrix, 0.7, criterion="distance")
        columns = self.corr_matrix.columns.tolist()
        return dict(zip(columns, labels))

    def get_cluster_summary(self, cluster_labels):
        df = pd.DataFrame({"column": list(cluster_labels.keys()),
                           "cluster": list(cluster_labels.values())})
        summaries = []
        for cluster_id in df["cluster"].unique():
            members = df[df["cluster"] == cluster_id]["column"].tolist()
            if len(members) > 1:
                sub_corr = self.corr_matrix.loc[members, members]
                mask = ~np.eye(len(members), dtype=bool)
                avg_corr = sub_corr.values[mask].mean()
            else:
                avg_corr = 1.0
            summaries.append({
                "cluster": cluster_id, "n_members": len(members),
                "avg_internal_corr": avg_corr,
                "members": ", ".join(members[:5]) + ("..." if len(members) > 5 else "")
            })
        return pd.DataFrame(summaries).sort_values("cluster")


class CorrelationAnalysisSuite:
    """상관관계 분석 통합 클래스"""

    def __init__(self, data):
        self.data = data.copy()
        self._static_analyzer = StaticCorrelationAnalyzer(data)
        self._rolling_analyzer = RollingCorrelationAnalyzer(data)
        self._cross_analyzer = CrossCorrelationAnalyzer(data)

    def static_correlation(self, method="pearson"):
        return self._static_analyzer.calculate(method)

    def rolling_correlation(self, col1, col2, window=None):
        return self._rolling_analyzer.calculate_pairwise(col1, col2, window)

    def multi_window_correlation(self, col1, col2):
        return self._rolling_analyzer.calculate_multi_window(col1, col2)

    def cross_correlation(self, col1, col2, max_lag=None):
        return self._cross_analyzer.calculate(col1, col2, max_lag)

    def lead_lag_matrix(self, columns=None):
        return self._cross_analyzer.analyze_lead_lag_matrix(columns)

    def cluster_analysis(self, n_clusters=None, method="ward"):
        static = self.static_correlation()
        clusterer = CorrelationClusterer(static.correlation_matrix)
        labels = clusterer.hierarchical_clustering(method, n_clusters)
        summary = clusterer.get_cluster_summary(labels)
        return labels, summary

    def comprehensive_analysis(self, pairs=None):
        results = {}
        static = self.static_correlation()
        results["static_correlation"] = static
        results["significant_pairs"] = static.get_significant_pairs()
        labels, summary = self.cluster_analysis()
        results["clusters"] = labels
        results["cluster_summary"] = summary
        if pairs is None:
            sig_pairs = static.get_significant_pairs(threshold=0.8)
            if not sig_pairs.empty:
                pairs = list(zip(sig_pairs["pair_1"][:5], sig_pairs["pair_2"][:5]))
            else:
                pairs = []
        pair_analyses = {}
        for col1, col2 in pairs:
            pair_analyses[f"{col1}_vs_{col2}"] = {
                "rolling": self.multi_window_correlation(col1, col2),
                "cross_corr": self.cross_correlation(col1, col2)
            }
        results["pair_analyses"] = pair_analyses
        return results


# ============================================================================
# Part 4: 공적분 검정 및 페어트레이딩
# ============================================================================

class SignalType(Enum):
    LONG_SPREAD = "long_spread"
    SHORT_SPREAD = "short_spread"
    EXIT = "exit"
    HOLD = "hold"


@dataclass
class CointegrationTestResult:
    series1_name: str
    series2_name: str
    test_statistic: float
    p_value: float
    critical_values: Dict[str, float]
    alpha: float
    beta: float
    is_cointegrated: bool
    confidence_level: str


@dataclass
class SpreadAnalysis:
    spread: pd.Series
    zscore: pd.Series
    mean: float
    std: float
    current_zscore: float
    half_life: Optional[float] = None
    current_signal: SignalType = SignalType.HOLD


def adf_test(series, max_lags=None, significance=0.05):
    """ADF 단위근 검정"""
    if not HAS_STATSMODELS:
        raise ImportError("statsmodels가 필요합니다.")
    series = series.dropna()
    if max_lags is None:
        max_lags = ANALYSIS_CONFIG.adf_max_lags
    result = adfuller(series, maxlag=max_lags, autolag="AIC")
    test_stat, p_value, used_lag, nobs, crit_values, icbest = result
    is_stationary = p_value < significance
    if p_value < 0.01:
        confidence = "1%"
    elif p_value < 0.05:
        confidence = "5%"
    elif p_value < 0.10:
        confidence = "10%"
    else:
        confidence = "not significant"
    return {
        "test_statistic": test_stat, "p_value": p_value,
        "used_lag": used_lag, "n_observations": nobs,
        "critical_values": dict(crit_values),
        "is_stationary": is_stationary, "confidence": confidence
    }


class CointegrationTester:
    def __init__(self, significance=None):
        if not HAS_STATSMODELS:
            raise ImportError("statsmodels가 필요합니다.")
        self.significance = significance or ANALYSIS_CONFIG.coint_significance

    def test(self, series1, series2):
        common_idx = series1.dropna().index.intersection(series2.dropna().index)
        y, x = series1.loc[common_idx].values, series2.loc[common_idx].values
        X = add_constant(x)
        model = OLS(y, X).fit()
        alpha, beta = model.params[0], model.params[1]
        residuals = y - (alpha + beta * x)
        adf_result = adf_test(pd.Series(residuals), significance=self.significance)
        return CointegrationTestResult(
            series1_name=series1.name or "Series1",
            series2_name=series2.name or "Series2",
            test_statistic=adf_result["test_statistic"],
            p_value=adf_result["p_value"],
            critical_values=adf_result["critical_values"],
            alpha=alpha, beta=beta,
            is_cointegrated=adf_result["is_stationary"],
            confidence_level=adf_result["confidence"]
        )

    def test_multiple(self, data, columns=None):
        if columns is None:
            columns = data.columns.tolist()
        n = len(columns)
        results = pd.DataFrame(np.ones((n, n)), index=columns, columns=columns)
        for i in range(n):
            for j in range(i + 1, n):
                try:
                    result = self.test(data[columns[i]], data[columns[j]])
                    results.iloc[i, j] = results.iloc[j, i] = result.p_value
                except Exception:
                    results.iloc[i, j] = results.iloc[j, i] = np.nan
        return results

    def find_cointegrated_pairs(self, data, columns=None):
        if columns is None:
            columns = data.columns.tolist()
        pairs = []
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                try:
                    result = self.test(data[columns[i]], data[columns[j]])
                    if result.is_cointegrated:
                        pairs.append((columns[i], columns[j], result))
                except Exception:
                    continue
        pairs.sort(key=lambda x: x[2].p_value)
        return pairs


class SpreadAnalyzer:
    def __init__(self, lookback=None, entry_threshold=None, exit_threshold=None):
        self.lookback = lookback or ANALYSIS_CONFIG.lookback_period
        self.entry_threshold = entry_threshold or ANALYSIS_CONFIG.zscore_entry_threshold
        self.exit_threshold = exit_threshold or ANALYSIS_CONFIG.zscore_exit_threshold

    def calculate_spread(self, series1, series2, beta=None):
        if beta is None:
            beta = 1.0
        common_idx = series1.dropna().index.intersection(series2.dropna().index)
        spread = series1.loc[common_idx] - beta * series2.loc[common_idx]
        spread.name = f"Spread({series1.name}, {series2.name})"
        return spread

    def analyze(self, series1, series2, coint_result=None):
        if coint_result is not None:
            beta, alpha = coint_result.beta, coint_result.alpha
        else:
            beta, alpha = 1.0, 0.0
        common_idx = series1.dropna().index.intersection(series2.dropna().index)
        spread = series1.loc[common_idx] - alpha - beta * series2.loc[common_idx]
        spread.name = "Spread"
        rolling_mean = spread.rolling(window=self.lookback).mean()
        rolling_std = spread.rolling(window=self.lookback).std()
        zscore = (spread - rolling_mean) / rolling_std
        zscore.name = "Z-score"
        current_zscore = zscore.iloc[-1] if not zscore.empty else np.nan
        half_life = self._calculate_half_life(spread)
        current_signal = self._determine_signal(current_zscore)
        return SpreadAnalysis(
            spread=spread, zscore=zscore, mean=spread.mean(),
            std=spread.std(), current_zscore=current_zscore,
            half_life=half_life, current_signal=current_signal
        )

    def _calculate_half_life(self, spread):
        try:
            spread = spread.dropna()
            if len(spread) < 20:
                return None
            y = spread.diff().dropna()
            x = spread.shift(1).dropna()
            common_idx = y.index.intersection(x.index)
            y, x = y.loc[common_idx], x.loc[common_idx] - spread.mean()
            X = add_constant(x)
            model = OLS(y, X).fit()
            lambda_param = model.params[1]
            if lambda_param >= 0:
                return None
            half_life = -np.log(2) / lambda_param
            return half_life if half_life > 0 else None
        except Exception:
            return None

    def _determine_signal(self, zscore):
        if np.isnan(zscore):
            return SignalType.HOLD
        if zscore > self.entry_threshold:
            return SignalType.SHORT_SPREAD
        elif zscore < -self.entry_threshold:
            return SignalType.LONG_SPREAD
        elif abs(zscore) < self.exit_threshold:
            return SignalType.EXIT
        return SignalType.HOLD

    def generate_signals(self, spread_analysis):
        zscore = spread_analysis.zscore.dropna()
        spread = spread_analysis.spread
        signals = []
        for date in zscore.index:
            z = zscore[date]
            signal_type = self._determine_signal(z)
            if signal_type == SignalType.LONG_SPREAD:
                pos1, pos2, reason = "long", "short", f"Z ({z:.2f}) < -{self.entry_threshold}"
            elif signal_type == SignalType.SHORT_SPREAD:
                pos1, pos2, reason = "short", "long", f"Z ({z:.2f}) > {self.entry_threshold}"
            elif signal_type == SignalType.EXIT:
                pos1, pos2, reason = "exit", "exit", f"|Z| ({abs(z):.2f}) < {self.exit_threshold}"
            else:
                pos1, pos2, reason = "hold", "hold", "No signal"
            signals.append({"date": date, "signal": signal_type.value,
                            "zscore": z, "spread": spread.get(date, np.nan),
                            "position_1": pos1, "position_2": pos2, "reason": reason})
        return pd.DataFrame(signals).set_index("date")


class PairTradingStrategy:
    """페어트레이딩 전략 통합"""

    def __init__(self, data, lookback=None, entry_threshold=None, exit_threshold=None):
        self.data = data.copy()
        self.coint_tester = CointegrationTester()
        self.spread_analyzer = SpreadAnalyzer(lookback, entry_threshold, exit_threshold)

    def analyze_pair(self, col1, col2):
        series1, series2 = self.data[col1], self.data[col2]
        coint_result = self.coint_tester.test(series1, series2)
        spread_analysis = self.spread_analyzer.analyze(series1, series2, coint_result)
        signals = self.spread_analyzer.generate_signals(spread_analysis)
        summary = {
            "pair": f"{col1} vs {col2}",
            "is_cointegrated": coint_result.is_cointegrated,
            "p_value": coint_result.p_value,
            "hedge_ratio": coint_result.beta,
            "half_life": spread_analysis.half_life,
            "current_zscore": spread_analysis.current_zscore,
            "current_signal": spread_analysis.current_signal.value,
            "n_entry_signals": len(signals[signals["signal"].isin(["long_spread", "short_spread"])])
        }
        return {"cointegration": coint_result, "spread_analysis": spread_analysis,
                "signals": signals, "summary": summary}

    def find_opportunities(self, columns=None, require_cointegration=True,
                           min_half_life=5, max_half_life=60):
        if columns is None:
            columns = self.data.columns.tolist()
        opportunities = []
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                try:
                    result = self.analyze_pair(columns[i], columns[j])
                    summary = result["summary"]
                    if require_cointegration and not summary["is_cointegrated"]:
                        continue
                    hl = summary["half_life"]
                    if hl is None or hl < min_half_life or hl > max_half_life:
                        continue
                    opportunities.append(summary)
                except Exception:
                    continue
        df = pd.DataFrame(opportunities)
        if not df.empty:
            df = df.sort_values("half_life")
        return df


# ============================================================================
# Part 5: 시각화 유틸리티
# ============================================================================

def get_diverging_cmap(center_color="white", low_color="#3b4cc0", high_color="#b40426"):
    return LinearSegmentedColormap.from_list("diverging", [low_color, center_color, high_color], N=256)


def format_axis_date(ax, rotation=45):
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=rotation, ha="right")


def plot_correlation_heatmap(corr_matrix, title="Correlation Matrix", figsize=None,
                              annot=True, cmap=None, vmin=-1, vmax=1,
                              mask_diagonal=True, save_path=None):
    if figsize is None:
        figsize = VIZ_CONFIG.figsize_heatmap
    if cmap is None or cmap == "diverging":
        cmap = get_diverging_cmap()
    fig, ax = plt.subplots(figsize=figsize)
    data = corr_matrix.values.copy()
    if mask_diagonal:
        np.fill_diagonal(data, np.nan)
    im = ax.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax, aspect="auto")
    n = len(corr_matrix)
    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels(corr_matrix.columns, rotation=45, ha="right")
    ax.set_yticklabels(corr_matrix.index)
    if annot and n <= 15:
        for i in range(n):
            for j in range(n):
                if mask_diagonal and i == j:
                    continue
                val = corr_matrix.iloc[i, j]
                color = "white" if abs(val) > 0.5 else "black"
                ax.text(j, i, f"{val:.2f}", ha="center", va="center",
                        color=color, fontsize=VIZ_CONFIG.font_size_annotation)
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Correlation", fontsize=VIZ_CONFIG.font_size_label)
    ax.set_title(title, fontsize=VIZ_CONFIG.font_size_title, pad=15)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight")
    return fig, ax


def plot_time_series(data, columns=None, title="Interest Rates", ylabel="Rate (%)",
                     figsize=None, colors=None, highlight_periods=None, save_path=None):
    if figsize is None:
        figsize = VIZ_CONFIG.figsize_single
    if columns is None:
        columns = data.columns.tolist()
    if colors is None:
        colors = list(VIZ_CONFIG.line_colors)
    fig, ax = plt.subplots(figsize=figsize)
    for i, col in enumerate(columns):
        ax.plot(data.index, data[col], label=col, color=colors[i % len(colors)],
                linewidth=VIZ_CONFIG.linewidth_main)
    if highlight_periods:
        for start, end, label in highlight_periods:
            ax.axvspan(start, end, alpha=0.2, color="gray", label=label)
    format_axis_date(ax)
    ax.set_ylabel(ylabel, fontsize=VIZ_CONFIG.font_size_label)
    ax.set_title(title, fontsize=VIZ_CONFIG.font_size_title)
    ax.legend(loc=VIZ_CONFIG.legend_loc, fontsize=VIZ_CONFIG.font_size_legend,
              frameon=VIZ_CONFIG.legend_frameon, framealpha=VIZ_CONFIG.legend_framealpha)
    ax.grid(True, alpha=VIZ_CONFIG.grid_alpha, linestyle=VIZ_CONFIG.grid_linestyle,
            linewidth=VIZ_CONFIG.grid_linewidth)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight")
    return fig, ax


# ============================================================================
# Part 6: 확장 데이터 로더 (OHLCV, 선물)
# ============================================================================

class BondOHLCVLoader:
    """국고지표정보 파서"""

    def __init__(self, config=None):
        self.config = config or OHLCV_CONFIG
        self._data = {}

    def load_from_xlsb(self, file_path=None):
        path = Path(file_path) if file_path else DATA_CONFIG.file_path
        if not path.exists():
            print(f"[WARNING] 파일 없음: {path}")
            return {}
        print(f"  국고지표 로딩: {path.name} → {self.config.sheet_name}")
        raw_data = read_xlsb_raw(path, self.config.sheet_name)
        n_rows = len(raw_data)
        n_cols = len(raw_data[0]) if raw_data else 0
        print(f"  시트: {n_rows}행 × {n_cols}열")
        result = self._parse(raw_data)
        self._data = result
        if result:
            print(f"  국고지표 OHLCV 로딩: {len(result)}종목")
        return result

    def _parse(self, raw_data):
        data_start = self.config.data_start_row - 1
        dates, valid_row_indices = [], []
        for idx, row in enumerate(raw_data[data_start:], start=data_start):
            val = row[0]
            if val is None:
                continue
            dt = excel_serial_to_datetime(val)
            if dt is None:
                continue
            dates.append(pd.Timestamp(dt))
            valid_row_indices.append(idx)

        n_rows = len(dates)
        if n_rows == 0:
            return {}

        last_idx = valid_row_indices[-1]
        last_row = raw_data[last_idx]
        close_offset = self.config.col_map_first.get('close', 10)
        if close_offset < len(last_row) and last_row[close_offset] is None:
            n_rows -= 1
            dates = dates[:n_rows]
            valid_row_indices = valid_row_indices[:n_rows]
        if n_rows == 0:
            return {}

        if len(dates) > 1 and dates[0] > dates[-1]:
            dates = list(reversed(dates))
            valid_row_indices = list(reversed(valid_row_indices))

        date_index = pd.DatetimeIndex(dates)
        data_rows = [raw_data[i] for i in valid_row_indices]

        result = {}
        for name, col_start in self.config.instruments.items():
            is_first = (col_start == min(self.config.instruments.values()))
            col_map = self.config.col_map_first if is_first else self.config.col_map_rest
            df = pd.DataFrame(index=date_index)
            df.index.name = '일자'
            for field_name, offset in col_map.items():
                col_idx = (col_start - 1) + offset
                values = []
                for row in data_rows:
                    if col_idx < len(row):
                        v = row[col_idx]
                        values.append(None if isinstance(v, str) else v)
                    else:
                        values.append(None)
                df[field_name] = pd.to_numeric(pd.Series(values, index=date_index), errors='coerce')
            if 'close' in df.columns and 'close_eval' in df.columns:
                df['close'] = df['close'].fillna(df['close_eval'])
            if 'net_volume' in df.columns:
                df['volume'] = df['net_volume'].abs()
            result[name] = df

        if n_rows > 0:
            print(f"  기간: {date_index[0].date()} ~ {date_index[-1].date()}, {n_rows}일")
        return result

    def get(self, name):
        return self._data.get(name)

    def list_instruments(self):
        return list(self._data.keys())


class FuturesLoader:
    """국고선물정보 파서"""

    def __init__(self, config=None):
        self.config = config or FUTURES_CONFIG
        self._data = {}

    def load_from_xlsb(self, file_path=None):
        path = Path(file_path) if file_path else DATA_CONFIG.file_path
        if not path.exists():
            print(f"[WARNING] 파일 없음: {path}")
            return {}
        print(f"  국고선물 로딩: {path.name} → {self.config.sheet_name}")
        raw_data = read_xlsb_raw(path, self.config.sheet_name)
        n_rows = len(raw_data)
        n_cols = len(raw_data[0]) if raw_data else 0
        print(f"  시트: {n_rows}행 × {n_cols}열")
        result = self._parse(raw_data)
        self._data = result
        if result:
            print(f"  국고선물 OHLCV 로딩: {len(result)}종목")
        return result

    def _parse(self, raw_data):
        data_start = self.config.data_start_row - 1
        dates, valid_row_indices = [], []
        for idx, row in enumerate(raw_data[data_start:], start=data_start):
            val = row[0]
            if val is None:
                continue
            dt = excel_serial_to_datetime(val)
            if dt is None:
                continue
            dates.append(pd.Timestamp(dt))
            valid_row_indices.append(idx)

        n_rows = len(dates)
        if n_rows == 0:
            return {}

        last_idx = valid_row_indices[-1]
        last_row = raw_data[last_idx]
        close_offset = self.config.col_map_first.get('close', 1)
        if close_offset < len(last_row) and last_row[close_offset] is None:
            n_rows -= 1
            dates = dates[:n_rows]
            valid_row_indices = valid_row_indices[:n_rows]
        if n_rows == 0:
            return {}

        if len(dates) > 1 and dates[0] > dates[-1]:
            dates = list(reversed(dates))
            valid_row_indices = list(reversed(valid_row_indices))

        date_index = pd.DatetimeIndex(dates)
        data_rows = [raw_data[i] for i in valid_row_indices]

        result = {}
        for name, col_start in self.config.instruments.items():
            is_first = (col_start == min(self.config.instruments.values()))
            col_map = self.config.col_map_first if is_first else self.config.col_map_rest
            df = pd.DataFrame(index=date_index)
            df.index.name = '일자'
            for field_name, offset in col_map.items():
                col_idx = (col_start - 1) + offset
                values = []
                for row in data_rows:
                    if col_idx < len(row):
                        v = row[col_idx]
                        values.append(None if isinstance(v, str) else v)
                    else:
                        values.append(None)
                df[field_name] = pd.to_numeric(pd.Series(values, index=date_index), errors='coerce')
            if 'close' in df.columns and 'settle' in df.columns:
                df['close'] = df['close'].fillna(df['settle'])
            result[name] = df

        if n_rows > 0:
            print(f"  기간: {date_index[0].date()} ~ {date_index[-1].date()}, {n_rows}일")
        return result

    def get(self, name):
        return self._data.get(name)

    def list_instruments(self):
        return list(self._data.keys())


@dataclass
class DataTier:
    tier: str
    has_ohlc: bool
    has_volume: bool
    source: str
    available_indicators: int


class DataTierDetector:
    """종목별 데이터 가용성 판별"""

    def __init__(self, bond_ohlcv_loader, futures_loader):
        self.bond_ohlcv = bond_ohlcv_loader
        self.futures = futures_loader
        self.futures_map = {
            '3년국채 연결': '국고 3Y', '5년국채 연결': '국고 5Y',
            '10년국채 연결': '국고 10Y', '30년국채 연결': '국고 30Y',
        }

    def detect(self, name):
        for fut_name in self.futures.list_instruments():
            if fut_name == name or self.futures_map.get(fut_name) == name:
                return DataTier(tier='1a', has_ohlc=True, has_volume=True,
                                source='futures', available_indicators=29)
        if name in self.bond_ohlcv.list_instruments():
            return DataTier(tier='1b', has_ohlc=True, has_volume=True,
                            source='bond_ohlcv', available_indicators=25)
        return DataTier(tier='2', has_ohlc=False, has_volume=False,
                        source='bond_close', available_indicators=13)

    def get_ohlcv(self, name, mode='auto', source_preference=None):
        """
        종목의 OHLCV 데이터 반환 (통합 인터페이스)

        Args:
            name: 종목명
            mode: 'auto' | 'price' | 'yield'
            source_preference: 'bond_ohlcv' | 'futures' | None
                               None이면 detect() 결과 사용
        """
        # source_preference가 지정되면 detect() 우회
        if source_preference == 'bond_ohlcv':
            raw = self.bond_ohlcv.get(name)
            if raw is not None:
                df = pd.DataFrame(index=raw.index)
                df['open'] = raw['open']
                df['high'] = raw['high']
                df['low'] = raw['low']
                df['close'] = raw['close']
                df['volume'] = raw.get('volume', pd.Series(dtype=float))
                return df
            return None

        tier = self.detect(name)

        if tier.source == 'futures':
            fut_name = name
            for fn, sn in self.futures_map.items():
                if sn == name:
                    fut_name = fn
                    break
            raw = self.futures.get(fut_name)
            if raw is None:
                return None
            if mode == 'yield':
                df = pd.DataFrame(index=raw.index)
                df['close'] = raw['yield']
                df['open'] = df['close']
                df['high'] = df['close']
                df['low'] = df['close']
                df['volume'] = raw.get('volume', pd.Series(dtype=float))
                return df
            else:
                df = pd.DataFrame(index=raw.index)
                df['open'] = raw['open']
                df['high'] = raw['high']
                df['low'] = raw['low']
                df['close'] = raw['close']
                df['volume'] = raw.get('volume', pd.Series(dtype=float))
                return df
        elif tier.source == 'bond_ohlcv':
            raw = self.bond_ohlcv.get(name)
            if raw is None:
                return None
            df = pd.DataFrame(index=raw.index)
            df['open'] = raw['open']
            df['high'] = raw['high']
            df['low'] = raw['low']
            df['close'] = raw['close']
            df['volume'] = raw.get('volume', pd.Series(dtype=float))
            return df
        return None

    def summary(self):
        rows = []
        all_names = list(self.bond_ohlcv.list_instruments())
        for fn in self.futures.list_instruments():
            mapped = self.futures_map.get(fn, fn)
            if mapped not in all_names:
                all_names.append(mapped)
        for name in sorted(all_names):
            t = self.detect(name)
            rows.append({
                '종목': name, '티어': t.tier, 'OHLC': t.has_ohlc,
                '거래량': t.has_volume, '소스': t.source, '지표수': t.available_indicators
            })
        return pd.DataFrame(rows)


# ============================================================================
# Part 7: 가격 + 추세 지표 엔진
# ============================================================================

class PriceTrendEngine:
    """가격 및 추세 지표 계산 엔진"""

    def __init__(self, config=None):
        self.cfg = config or INDICATOR_CONFIG

    def sma(self, series, period=None):
        n = period or self.cfg.sma_periods[2]
        return series.rolling(window=n, min_periods=1).mean()

    def ema(self, series, period=20):
        return series.ewm(span=period, adjust=False).mean()

    def dema(self, series, period=None):
        n = period or self.cfg.dema_period
        ema1 = self.ema(series, n)
        ema2 = self.ema(ema1, n)
        return 2 * ema1 - ema2

    def t3(self, series, period=None, vfactor=None):
        n = period or self.cfg.t3_period
        v = vfactor or self.cfg.t3_vfactor
        def _gd(s):
            e1 = self.ema(s, n)
            e2 = self.ema(e1, n)
            return e1 * (1 + v) - e2 * v
        return _gd(_gd(_gd(series)))

    def vidya(self, series, period=None):
        n = period or self.cfg.vidya_period
        diff = series.diff()
        up = diff.clip(lower=0).rolling(n).sum()
        down = (-diff.clip(upper=0)).rolling(n).sum()
        cmo = ((up - down) / (up + down)).abs()
        sc = 2.0 / (n + 1)
        result = series.copy()
        for i in range(n, len(series)):
            if pd.notna(cmo.iloc[i]):
                alpha = sc * cmo.iloc[i]
                result.iloc[i] = alpha * series.iloc[i] + (1 - alpha) * result.iloc[i - 1]
        return result

    def vwma(self, close, volume, period=None):
        n = period or self.cfg.sma_periods[2]
        vol_price = (close * volume).rolling(n, min_periods=1).sum()
        vol_sum = volume.rolling(n, min_periods=1).sum()
        return vol_price / vol_sum.replace(0, np.nan)

    def bollinger_bands(self, series, period=None, std_mult=None):
        n = period or self.cfg.bb_period
        k = std_mult or self.cfg.bb_std
        mid = series.rolling(n, min_periods=1).mean()
        std = series.rolling(n, min_periods=1).std()
        return {
            'middle': mid, 'upper': mid + k * std, 'lower': mid - k * std,
            'bandwidth': (2 * k * std) / mid * 100,
            'pctb': (series - (mid - k * std)) / (2 * k * std),
        }

    def ichimoku(self, high, low, close):
        t, k, sb = self.cfg.ichimoku_tenkan, self.cfg.ichimoku_kijun, self.cfg.ichimoku_senkou_b
        tenkan = (high.rolling(t).max() + low.rolling(t).min()) / 2
        kijun = (high.rolling(k).max() + low.rolling(k).min()) / 2
        senkou_a = ((tenkan + kijun) / 2).shift(k)
        senkou_b = ((high.rolling(sb).max() + low.rolling(sb).min()) / 2).shift(k)
        chikou = close.shift(-k)
        return {'tenkan': tenkan, 'kijun': kijun, 'senkou_a': senkou_a,
                'senkou_b': senkou_b, 'chikou': chikou}

    def parabolic_sar(self, high, low):
        af_init, af_step, af_max = self.cfg.sar_af_init, self.cfg.sar_af_step, self.cfg.sar_af_max
        n = len(high)
        sar = pd.Series(np.nan, index=high.index)
        af = af_init
        is_long = True
        ep = low.iloc[0]
        sar_val = high.iloc[0]
        for i in range(2, n):
            if is_long:
                sar_val = sar_val + af * (ep - sar_val)
                sar_val = min(sar_val, low.iloc[i - 1], low.iloc[i - 2])
                if low.iloc[i] < sar_val:
                    is_long, sar_val, ep, af = False, ep, low.iloc[i], af_init
                elif high.iloc[i] > ep:
                    ep = high.iloc[i]
                    af = min(af + af_step, af_max)
            else:
                sar_val = sar_val + af * (ep - sar_val)
                sar_val = max(sar_val, high.iloc[i - 1], high.iloc[i - 2])
                if high.iloc[i] > sar_val:
                    is_long, sar_val, ep, af = True, ep, high.iloc[i], af_init
                elif low.iloc[i] < ep:
                    ep = low.iloc[i]
                    af = min(af + af_step, af_max)
            sar.iloc[i] = sar_val
        return sar

    def dmi(self, high, low, close, period=None):
        n = period or self.cfg.dmi_period
        tr1 = high - low
        tr2 = (high - close.shift(1)).abs()
        tr3 = (low - close.shift(1)).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        up_move = high - high.shift(1)
        down_move = low.shift(1) - low
        plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0), index=high.index)
        minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0), index=high.index)
        atr = tr.ewm(alpha=1/n, adjust=False).mean()
        plus_di = 100 * plus_dm.ewm(alpha=1/n, adjust=False).mean() / atr
        minus_di = 100 * minus_dm.ewm(alpha=1/n, adjust=False).mean() / atr
        dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
        adx = dx.ewm(alpha=1/n, adjust=False).mean()
        return {'plus_di': plus_di, 'minus_di': minus_di, 'adx': adx, 'dx': dx}

    def cci(self, high, low, close, period=None):
        n = period or self.cfg.cci_period
        tp = (high + low + close) / 3
        ma = tp.rolling(n).mean()
        md = tp.rolling(n).apply(lambda x: np.abs(x - x.mean()).mean(), raw=True)
        return (tp - ma) / (0.015 * md)

    def macd(self, series, fast=None, slow=None, signal=None):
        f = fast or self.cfg.macd_fast
        s = slow or self.cfg.macd_slow
        sig = signal or self.cfg.macd_signal
        ema_fast = self.ema(series, f)
        ema_slow = self.ema(series, s)
        macd_line = ema_fast - ema_slow
        signal_line = self.ema(macd_line, sig)
        return {'macd': macd_line, 'signal': signal_line, 'histogram': macd_line - signal_line}

    def sonar(self, series, period=None):
        n = period or self.cfg.sonar_period
        e = self.ema(series, n)
        return e - e.shift(n)

    def compute_all(self, df, tier='1a'):
        result = {}
        close = df['close']
        for n in self.cfg.sma_periods:
            result[f'SMA_{n}'] = self.sma(close, n)
        result['DEMA'] = self.dema(close)
        result['T3'] = self.t3(close)
        result['VIDYA'] = self.vidya(close)
        macd_r = self.macd(close)
        result['MACD'] = macd_r['macd']
        result['MACD_signal'] = macd_r['signal']
        result['MACD_hist'] = macd_r['histogram']
        result['SONAR'] = self.sonar(close)
        bb = self.bollinger_bands(close)
        result['BB_upper'] = bb['upper']
        result['BB_middle'] = bb['middle']
        result['BB_lower'] = bb['lower']
        result['BB_bandwidth'] = bb['bandwidth']
        result['BB_pctb'] = bb['pctb']
        if tier in ('1a', '1b'):
            high, low = df['high'], df['low']
            ichi = self.ichimoku(high, low, close)
            for k, v in ichi.items():
                result[f'Ichimoku_{k}'] = v
            result['Parabolic_SAR'] = self.parabolic_sar(high, low)
            dmi_r = self.dmi(high, low, close)
            for k, v in dmi_r.items():
                result[f'DMI_{k}'] = v
            result['CCI'] = self.cci(high, low, close)
            if 'volume' in df.columns and df['volume'].notna().any():
                result['VWMA'] = self.vwma(close, df['volume'])
        return result


# ============================================================================
# Part 8: 변동성 + 모멘텀 지표 엔진
# ============================================================================

class VolatilityMomentumEngine:
    """변동성 및 모멘텀 지표 계산 엔진"""

    def __init__(self, config=None):
        self.cfg = config or INDICATOR_CONFIG

    def true_range(self, high, low, close):
        tr1 = high - low
        tr2 = (high - close.shift(1)).abs()
        tr3 = (low - close.shift(1)).abs()
        return pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    def atr(self, high, low, close, period=None):
        n = period or self.cfg.atr_period
        return self.true_range(high, low, close).ewm(alpha=1/n, adjust=False).mean()

    def chaikin_volatility(self, high, low, ema_period=None, roc_period=None):
        ep = ema_period or self.cfg.chaikin_ema_period
        rp = roc_period or self.cfg.chaikin_roc_period
        hl_ema = (high - low).ewm(span=ep, adjust=False).mean()
        return ((hl_ema - hl_ema.shift(rp)) / hl_ema.shift(rp).replace(0, np.nan)) * 100

    def stochastic(self, high, low, close, k_period=None, d_period=None, smooth=None):
        kp = k_period or self.cfg.stoch_k
        dp = d_period or self.cfg.stoch_d
        sm = smooth or self.cfg.stoch_smooth
        lowest = low.rolling(kp).min()
        highest = high.rolling(kp).max()
        fast_k = ((close - lowest) / (highest - lowest).replace(0, np.nan)) * 100
        k = fast_k.rolling(sm).mean()
        d = k.rolling(dp).mean()
        return {'k': k, 'd': d, 'fast_k': fast_k}

    def rsi(self, series, period=None):
        n = period or self.cfg.rsi_period
        delta = series.diff()
        gain = delta.clip(lower=0)
        loss = (-delta.clip(upper=0))
        avg_gain = gain.ewm(alpha=1/n, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/n, adjust=False).mean()
        rs = avg_gain / avg_loss.replace(0, np.nan)
        return 100 - (100 / (1 + rs))

    def williams_r(self, high, low, close, period=None):
        n = period or self.cfg.williams_period
        highest = high.rolling(n).max()
        lowest = low.rolling(n).min()
        return ((highest - close) / (highest - lowest).replace(0, np.nan)) * -100

    def stochastic_close_only(self, close, k_period=None):
        kp = k_period or self.cfg.stoch_k
        dp, sm = self.cfg.stoch_d, self.cfg.stoch_smooth
        lowest = close.rolling(kp).min()
        highest = close.rolling(kp).max()
        fast_k = ((close - lowest) / (highest - lowest).replace(0, np.nan)) * 100
        k = fast_k.rolling(sm).mean()
        d = k.rolling(dp).mean()
        return {'k': k, 'd': d, 'fast_k': fast_k}

    def williams_r_close_only(self, close, period=None):
        n = period or self.cfg.williams_period
        highest = close.rolling(n).max()
        lowest = close.rolling(n).min()
        return ((highest - close) / (highest - lowest).replace(0, np.nan)) * -100

    def compute_all(self, df, tier='1a'):
        result = {}
        close = df['close']
        result['RSI'] = self.rsi(close)
        if tier in ('1a', '1b'):
            high, low = df['high'], df['low']
            result['ATR'] = self.atr(high, low, close)
            result['Chaikin_Vol'] = self.chaikin_volatility(high, low)
            stoch = self.stochastic(high, low, close)
            result['Stoch_K'] = stoch['k']
            result['Stoch_D'] = stoch['d']
            result['Williams_R'] = self.williams_r(high, low, close)
        else:
            stoch = self.stochastic_close_only(close)
            result['Stoch_K'] = stoch['k']
            result['Stoch_D'] = stoch['d']
            result['Williams_R'] = self.williams_r_close_only(close)
        return result


# ============================================================================
# Part 9: 거래량 + 시장강도 지표 엔진
# ============================================================================

class VolumeStrengthEngine:
    """거래량 및 시장강도 지표 계산 엔진"""

    def __init__(self, config=None):
        self.cfg = config or INDICATOR_CONFIG

    def obv(self, close, volume):
        direction = np.sign(close.diff())
        return (direction * volume).fillna(0).cumsum()

    def ad_line(self, high, low, close, volume):
        hl_range = (high - low).replace(0, np.nan)
        clv = ((close - low) - (high - close)) / hl_range
        return (clv * volume).fillna(0).cumsum()

    def chaikin_mf(self, high, low, close, volume, period=None):
        n = period or self.cfg.cmf_period
        hl_range = (high - low).replace(0, np.nan)
        clv = ((close - low) - (high - close)) / hl_range
        mfv = clv * volume
        return mfv.rolling(n).sum() / volume.rolling(n).sum().replace(0, np.nan)

    def compute_all(self, df, tier='1a'):
        result = {}
        if 'volume' not in df.columns or df['volume'].isna().all():
            return result
        close, volume = df['close'], df['volume']
        result['OBV'] = self.obv(close, volume)
        if tier in ('1a', '1b') and 'high' in df.columns:
            high, low = df['high'], df['low']
            result['AD_Line'] = self.ad_line(high, low, close, volume)
            result['CMF'] = self.chaikin_mf(high, low, close, volume)
        return result


# ============================================================================
# Part 10: 차트 렌더러 (특수차트 + 오버레이 + CompositeDashboard)
# ============================================================================

class SpecialChartRenderer:
    """특수 차트 렌더러"""

    def __init__(self):
        self.viz = VIZ_CONFIG

    def candlestick(self, df, ax=None, title='', is_yield=True):
        if ax is None:
            fig, ax = plt.subplots(figsize=(14, 7))
        dates = mdates.date2num(df.index.to_pydatetime())
        o, h, l, c = df['open'], df['high'], df['low'], df['close']
        for i in range(len(df)):
            if pd.isna(o.iloc[i]) or pd.isna(c.iloc[i]):
                continue
            if is_yield:
                color = self.viz.colors['positive'] if c.iloc[i] < o.iloc[i] else self.viz.colors['negative']
            else:
                color = self.viz.colors['positive'] if c.iloc[i] > o.iloc[i] else self.viz.colors['negative']
            body_bottom = min(o.iloc[i], c.iloc[i])
            body_height = abs(c.iloc[i] - o.iloc[i])
            rect = mpatches.FancyBboxPatch((dates[i] - 0.3, body_bottom), 0.6, body_height or 0.001,
                                            boxstyle="square,pad=0", facecolor=color, edgecolor='black', linewidth=0.5)
            ax.add_patch(rect)
            if not pd.isna(h.iloc[i]):
                ax.plot([dates[i], dates[i]], [body_bottom + body_height, h.iloc[i]], color='black', linewidth=0.5)
            if not pd.isna(l.iloc[i]):
                ax.plot([dates[i], dates[i]], [body_bottom, l.iloc[i]], color='black', linewidth=0.5)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        ax.set_xlim(dates[0] - 1, dates[-1] + 1)
        ax.set_ylim(l.min() * 0.999, h.max() * 1.001)
        ax.set_title(title, fontsize=self.viz.font_size_title)
        ax.grid(alpha=self.viz.grid_alpha)
        return ax

    def point_and_figure(self, close, box_size=None, reversal=None, ax=None, title=''):
        bs = box_size or INDICATOR_CONFIG.pnf_box_size
        rev = reversal or INDICATOR_CONFIG.pnf_reversal
        if ax is None:
            fig, ax = plt.subplots(figsize=(14, 7))
        columns = []
        prices = close.dropna().values
        if len(prices) < 2:
            return ax
        direction = 'X' if prices[1] > prices[0] else 'O'
        col_start = prices[0]
        col_end = prices[0]
        for p in prices[1:]:
            if direction == 'X':
                if p >= col_end + bs:
                    col_end = p
                elif p <= col_end - bs * rev:
                    columns.append((col_start, col_end, 'X'))
                    direction, col_start, col_end = 'O', col_end, p
            else:
                if p <= col_end - bs:
                    col_end = p
                elif p >= col_end + bs * rev:
                    columns.append((col_start, col_end, 'O'))
                    direction, col_start, col_end = 'X', col_end, p
        columns.append((col_start, col_end, direction))
        for i, (start, end, dtype) in enumerate(columns):
            n_boxes = int(abs(end - start) / bs) + 1
            base = min(start, end)
            for j in range(n_boxes):
                y = base + j * bs
                if dtype == 'X':
                    ax.plot(i, y, 'x', color=self.viz.colors['positive'], markersize=8)
                else:
                    ax.plot(i, y, 'o', color=self.viz.colors['negative'], markersize=6, markerfacecolor='none')
        ax.set_xlabel('Column')
        ax.set_title(title or 'Point & Figure', fontsize=self.viz.font_size_title)
        ax.grid(alpha=self.viz.grid_alpha)
        return ax

    def three_line_break(self, close, n_lines=None, ax=None, title=''):
        n = n_lines or INDICATOR_CONFIG.three_line_break_lines
        if ax is None:
            fig, ax = plt.subplots(figsize=(14, 7))
        prices = close.dropna().values
        if len(prices) < 2:
            return ax
        lines = [(prices[0], prices[1], 'up' if prices[1] > prices[0] else 'down')]
        for p in prices[2:]:
            last = lines[-1]
            if last[2] == 'up':
                if p > last[1]:
                    lines.append((last[1], p, 'up'))
                else:
                    lookback = [l for l in lines[-n:] if l[2] == 'up']
                    if lookback and p < min(l[0] for l in lookback):
                        lines.append((last[1], p, 'down'))
            else:
                if p < last[1]:
                    lines.append((last[1], p, 'down'))
                else:
                    lookback = [l for l in lines[-n:] if l[2] == 'down']
                    if lookback and p > max(l[0] for l in lookback):
                        lines.append((last[1], p, 'up'))
        for i, (o_val, c_val, d) in enumerate(lines):
            color = self.viz.colors['positive'] if d == 'down' else self.viz.colors['negative']
            bottom = min(o_val, c_val)
            height = abs(c_val - o_val)
            rect = mpatches.Rectangle((i - 0.4, bottom), 0.8, height or 0.001,
                                       facecolor=color, edgecolor='black', linewidth=0.5)
            ax.add_patch(rect)
        ax.set_xlim(-1, len(lines))
        if lines:
            all_prices = [v for l in lines for v in [l[0], l[1]]]
            ax.set_ylim(min(all_prices) * 0.999, max(all_prices) * 1.001)
        ax.set_title(title or '삼선전환도', fontsize=self.viz.font_size_title)
        ax.grid(alpha=self.viz.grid_alpha)
        return ax

    def counter_clockwise(self, close, volume, ax=None, title=''):
        if ax is None:
            fig, ax = plt.subplots(figsize=(8, 8))
        valid = pd.DataFrame({'price': close, 'volume': volume}).dropna()
        if len(valid) < 3:
            return ax
        p_ma = valid['price'].rolling(5, min_periods=1).mean()
        v_ma = valid['volume'].rolling(5, min_periods=1).mean()
        colors = plt.cm.viridis(np.linspace(0, 1, len(p_ma)))
        ax.scatter(v_ma, p_ma, c=colors, s=10, zorder=3)
        ax.plot(v_ma, p_ma, color='gray', alpha=0.3, linewidth=0.5)
        ax.scatter(v_ma.iloc[0], p_ma.iloc[0], color='blue', s=80, marker='^', zorder=4, label='시작')
        ax.scatter(v_ma.iloc[-1], p_ma.iloc[-1], color='red', s=80, marker='v', zorder=4, label='현재')
        ax.set_xlabel('거래량 (MA5)', fontsize=self.viz.font_size_label)
        ax.set_ylabel('가격/수익률 (MA5)', fontsize=self.viz.font_size_label)
        ax.set_title(title or '역시계곡선', fontsize=self.viz.font_size_title)
        ax.legend(fontsize=self.viz.font_size_legend)
        ax.grid(alpha=self.viz.grid_alpha)
        return ax

    def volume_profile(self, close, volume, n_bins=30, ax=None, title=''):
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 7))
        valid = pd.DataFrame({'close': close, 'volume': volume}).dropna()
        if len(valid) < 5:
            return ax
        bins = np.linspace(valid['close'].min(), valid['close'].max(), n_bins + 1)
        vol_by_price, centers = [], []
        for i in range(len(bins) - 1):
            mask = (valid['close'] >= bins[i]) & (valid['close'] < bins[i + 1])
            vol_by_price.append(valid.loc[mask, 'volume'].sum())
            centers.append((bins[i] + bins[i + 1]) / 2)
        ax.barh(centers, vol_by_price, height=(bins[1] - bins[0]) * 0.8,
                color=self.viz.colors['primary'], alpha=0.6, edgecolor='black', linewidth=0.3)
        poc_idx = np.argmax(vol_by_price)
        ax.barh(centers[poc_idx], vol_by_price[poc_idx], height=(bins[1] - bins[0]) * 0.8,
                color=self.viz.colors['negative'], alpha=0.8, label=f'POC: {centers[poc_idx]:.3f}')
        ax.set_ylabel('가격/수익률', fontsize=self.viz.font_size_label)
        ax.set_xlabel('거래량', fontsize=self.viz.font_size_label)
        ax.set_title(title or '볼륨 프로파일', fontsize=self.viz.font_size_title)
        ax.legend(fontsize=self.viz.font_size_legend)
        ax.grid(alpha=self.viz.grid_alpha)
        return ax

    def candle_volume(self, df, ax_price=None, ax_vol=None, title='', is_yield=True):
        if ax_price is None:
            fig, (ax_price, ax_vol) = plt.subplots(2, 1, figsize=(14, 9),
                                                     height_ratios=[3, 1], sharex=True)
        self.candlestick(df, ax=ax_price, title=title, is_yield=is_yield)
        if 'volume' in df.columns:
            colors = []
            for i in range(len(df)):
                if pd.isna(df['close'].iloc[i]) or pd.isna(df['open'].iloc[i]):
                    colors.append('gray')
                elif (is_yield and df['close'].iloc[i] < df['open'].iloc[i]) or \
                     (not is_yield and df['close'].iloc[i] > df['open'].iloc[i]):
                    colors.append(self.viz.colors['positive'])
                else:
                    colors.append(self.viz.colors['negative'])
            dates = mdates.date2num(df.index.to_pydatetime())
            ax_vol.bar(dates, df['volume'], width=0.6, color=colors, alpha=0.7)
            ax_vol.set_ylabel('거래량', fontsize=self.viz.font_size_label)
            ax_vol.grid(alpha=self.viz.grid_alpha)
        return ax_price, ax_vol


class IndicatorOverlay:
    """지표 오버레이 렌더러"""

    def __init__(self):
        self.viz = VIZ_CONFIG

    def overlay_ma(self, ax, ma_series, label, color=None, linestyle='-'):
        dates = mdates.date2num(ma_series.index.to_pydatetime())
        ax.plot(dates, ma_series, label=label, color=color or self.viz.colors['primary'],
                linewidth=self.viz.linewidth_secondary, linestyle=linestyle)

    def overlay_bollinger(self, ax, bb):
        dates = mdates.date2num(bb['middle'].index.to_pydatetime())
        ax.plot(dates, bb['middle'], label='BB Mid', color='gray', linewidth=0.8, linestyle='--')
        ax.fill_between(dates, bb['lower'], bb['upper'], alpha=0.1,
                        color=self.viz.colors['primary'], label='BB Band')

    def overlay_ichimoku(self, ax, ichi):
        dates = mdates.date2num(ichi['tenkan'].index.to_pydatetime())
        ax.plot(dates, ichi['tenkan'], label='전환선', color='blue', linewidth=0.8)
        ax.plot(dates, ichi['kijun'], label='기준선', color='red', linewidth=0.8)
        sa, sb = ichi['senkou_a'].dropna(), ichi['senkou_b'].dropna()
        common = sa.index.intersection(sb.index)
        if len(common) > 0:
            d = mdates.date2num(common.to_pydatetime())
            ax.fill_between(d, sa[common], sb[common], where=sa[common] >= sb[common],
                            alpha=0.15, color='green', label='양운')
            ax.fill_between(d, sa[common], sb[common], where=sa[common] < sb[common],
                            alpha=0.15, color='red', label='음운')

    def overlay_sar(self, ax, sar):
        dates = mdates.date2num(sar.index.to_pydatetime())
        ax.scatter(dates, sar, s=5, color='purple', alpha=0.6, label='SAR', zorder=3)


class CompositeDashboard:
    """종합 멀티패널 대시보드"""

    def __init__(self):
        self.viz = VIZ_CONFIG
        self.chart = SpecialChartRenderer()
        self.overlay = IndicatorOverlay()

    def render(self, df, indicators, overlays, subpanels, title='', is_yield=True, figsize=None):
        n_panels = 1 + len(subpanels)
        has_volume_panel = 'volume' in df.columns and df['volume'].notna().any()
        if has_volume_panel:
            n_panels += 1
        heights = [4]
        if has_volume_panel:
            heights.append(1)
        for _ in subpanels:
            heights.append(2)
        total_height = max(10, 3 * n_panels)
        fig_w = figsize[0] if figsize else 14
        fig_h = figsize[1] if figsize else total_height
        fig, axes = plt.subplots(n_panels, 1, figsize=(fig_w, fig_h),
                                  height_ratios=heights, sharex=True)
        if n_panels == 1:
            axes = [axes]
        ax_idx = 0

        ax_main = axes[ax_idx]
        dates = mdates.date2num(df.index.to_pydatetime())
        ax_main.plot(dates, df['close'], color='black', linewidth=1.2, label='Close')
        ma_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        ci = 0
        for ov in overlays:
            if ov.startswith('SMA_') and ov in indicators:
                self.overlay.overlay_ma(ax_main, indicators[ov], ov, ma_colors[ci % len(ma_colors)])
                ci += 1
            elif ov.startswith('DEMA') and 'DEMA' in indicators:
                self.overlay.overlay_ma(ax_main, indicators['DEMA'], 'DEMA', ma_colors[ci % len(ma_colors)], '--')
                ci += 1
            elif ov == 'T3' and 'T3' in indicators:
                self.overlay.overlay_ma(ax_main, indicators['T3'], 'T3', ma_colors[ci % len(ma_colors)], '-.')
                ci += 1
            elif ov == 'VIDYA' and 'VIDYA' in indicators:
                self.overlay.overlay_ma(ax_main, indicators['VIDYA'], 'VIDYA', ma_colors[ci % len(ma_colors)], ':')
                ci += 1
            elif ov == 'VWMA' and 'VWMA' in indicators:
                self.overlay.overlay_ma(ax_main, indicators['VWMA'], 'VWMA', ma_colors[ci % len(ma_colors)])
                ci += 1
            elif ov == 'Bollinger':
                bb = {k.replace('BB_', ''): indicators[k] for k in indicators if k.startswith('BB_')}
                if bb:
                    self.overlay.overlay_bollinger(ax_main, bb)
            elif ov == 'Ichimoku':
                ichi = {k.replace('Ichimoku_', ''): indicators[k] for k in indicators if k.startswith('Ichimoku_')}
                if ichi:
                    self.overlay.overlay_ichimoku(ax_main, ichi)
            elif ov == 'Parabolic_SAR' and 'Parabolic_SAR' in indicators:
                self.overlay.overlay_sar(ax_main, indicators['Parabolic_SAR'])

        ax_main.set_title(title, fontsize=self.viz.font_size_title, fontweight='bold')
        ax_main.set_ylabel('수익률 (%)' if is_yield else '가격', fontsize=self.viz.font_size_label)
        ax_main.legend(loc='upper left', fontsize=7, ncol=3)
        ax_main.grid(alpha=self.viz.grid_alpha)
        ax_idx += 1

        if has_volume_panel:
            ax_vol = axes[ax_idx]
            colors = []
            for i in range(len(df)):
                if i == 0 or pd.isna(df['close'].iloc[i]):
                    colors.append('gray')
                elif df['close'].iloc[i] < df['close'].iloc[i-1]:
                    colors.append(self.viz.colors['positive'])
                else:
                    colors.append(self.viz.colors['negative'])
            ax_vol.bar(dates, df['volume'].fillna(0), width=0.6, color=colors, alpha=0.7)
            ax_vol.set_ylabel('거래량', fontsize=8)
            ax_vol.grid(alpha=self.viz.grid_alpha)
            ax_idx += 1

        for panel_name in subpanels:
            ax = axes[ax_idx]
            self._render_subpanel(ax, panel_name, indicators, dates)
            ax_idx += 1

        axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        axes[-1].xaxis.set_major_locator(mdates.AutoDateLocator())
        plt.setp(axes[-1].xaxis.get_majorticklabels(), rotation=45)
        fig.tight_layout()
        return fig, axes

    def _render_subpanel(self, ax, name, indicators, dates):
        if name == 'MACD':
            if all(k in indicators for k in ['MACD', 'MACD_signal', 'MACD_hist']):
                ax.plot(dates, indicators['MACD'], label='MACD', color='blue', linewidth=1)
                ax.plot(dates, indicators['MACD_signal'], label='Signal', color='red', linewidth=1)
                colors = ['green' if v >= 0 else 'red' for v in indicators['MACD_hist']]
                ax.bar(dates, indicators['MACD_hist'], width=0.6, color=colors, alpha=0.5)
                ax.axhline(0, color='black', linewidth=0.5)
                ax.legend(loc='upper left', fontsize=7)
            ax.set_ylabel('MACD', fontsize=8)
        elif name == 'RSI':
            if 'RSI' in indicators:
                ax.plot(dates, indicators['RSI'], color='purple', linewidth=1)
                ax.axhline(INDICATOR_CONFIG.rsi_overbought, color='red', linestyle='--', linewidth=0.5,
                           label=f'{INDICATOR_CONFIG.rsi_overbought}')
                ax.axhline(INDICATOR_CONFIG.rsi_oversold, color='green', linestyle='--', linewidth=0.5,
                           label=f'{INDICATOR_CONFIG.rsi_oversold}')
                ax.fill_between(dates, INDICATOR_CONFIG.rsi_oversold,
                                INDICATOR_CONFIG.rsi_overbought, alpha=0.05, color='gray')
                ax.set_ylim(0, 100)
                ax.legend(loc='upper left', fontsize=6)
            ax.set_ylabel('RSI', fontsize=8)
        elif name == 'Stochastic':
            if 'Stoch_K' in indicators:
                ax.plot(dates, indicators['Stoch_K'], label='%K', color='blue', linewidth=1)
                ax.plot(dates, indicators['Stoch_D'], label='%D', color='red', linewidth=1)
                ax.axhline(80, color='red', linestyle='--', linewidth=0.5)
                ax.axhline(20, color='green', linestyle='--', linewidth=0.5)
                ax.set_ylim(0, 100)
                ax.legend(loc='upper left', fontsize=7)
            ax.set_ylabel('Stochastic', fontsize=8)
        elif name == 'Williams_R':
            if 'Williams_R' in indicators:
                ax.plot(dates, indicators['Williams_R'], color='brown', linewidth=1)
                ax.axhline(-20, color='red', linestyle='--', linewidth=0.5)
                ax.axhline(-80, color='green', linestyle='--', linewidth=0.5)
                ax.set_ylim(-100, 0)
            ax.set_ylabel('Williams %R', fontsize=8)
        elif name == 'DMI':
            if all(k in indicators for k in ['DMI_plus_di', 'DMI_minus_di', 'DMI_adx']):
                ax.plot(dates, indicators['DMI_plus_di'], label='+DI', color='green', linewidth=1)
                ax.plot(dates, indicators['DMI_minus_di'], label='-DI', color='red', linewidth=1)
                ax.plot(dates, indicators['DMI_adx'], label='ADX', color='black', linewidth=1.2)
                ax.axhline(25, color='gray', linestyle='--', linewidth=0.5)
                ax.legend(loc='upper left', fontsize=7)
            ax.set_ylabel('DMI', fontsize=8)
        elif name == 'CCI':
            if 'CCI' in indicators:
                ax.plot(dates, indicators['CCI'], color='teal', linewidth=1)
                ax.axhline(100, color='red', linestyle='--', linewidth=0.5)
                ax.axhline(-100, color='green', linestyle='--', linewidth=0.5)
                ax.axhline(0, color='black', linewidth=0.5)
            ax.set_ylabel('CCI', fontsize=8)
        elif name == 'SONAR':
            if 'SONAR' in indicators:
                ax.plot(dates, indicators['SONAR'], color='navy', linewidth=1)
                ax.axhline(0, color='black', linewidth=0.5)
            ax.set_ylabel('SONAR', fontsize=8)
        elif name == 'ATR':
            if 'ATR' in indicators:
                ax.plot(dates, indicators['ATR'], color='orange', linewidth=1)
            ax.set_ylabel('ATR', fontsize=8)
        elif name == 'Chaikin_Vol':
            if 'Chaikin_Vol' in indicators:
                ax.plot(dates, indicators['Chaikin_Vol'], color='brown', linewidth=1)
                ax.axhline(0, color='black', linewidth=0.5)
            ax.set_ylabel('Chaikin Vol', fontsize=8)
        elif name == 'OBV':
            if 'OBV' in indicators:
                ax.plot(dates, indicators['OBV'], color='teal', linewidth=1)
            ax.set_ylabel('OBV', fontsize=8)
        elif name == 'AD_Line':
            if 'AD_Line' in indicators:
                ax.plot(dates, indicators['AD_Line'], color='purple', linewidth=1)
            ax.set_ylabel('A/D Line', fontsize=8)
        elif name == 'CMF':
            if 'CMF' in indicators:
                colors = ['green' if v >= 0 else 'red' for v in indicators['CMF']]
                ax.bar(dates, indicators['CMF'], width=0.6, color=colors, alpha=0.6)
                ax.axhline(0, color='black', linewidth=0.5)
            ax.set_ylabel('CMF', fontsize=8)
        elif name == 'Bollinger_BW':
            if 'BB_bandwidth' in indicators:
                ax.plot(dates, indicators['BB_bandwidth'], color='gray', linewidth=1)
            ax.set_ylabel('BB Width', fontsize=8)
        ax.grid(alpha=self.viz.grid_alpha)


# ============================================================================
# Part 11: 시그널 분류 엔진
# ============================================================================

@dataclass
class IndicatorSignal:
    """개별 지표 시그널"""
    name: str
    category: str
    value: float
    signal: str
    description: str
    abs_gauge: float
    rel_gauges: Dict[str, float]
    direction: str
    divergence: bool


class SignalClassifier:
    """채권 관점 시그널 분류 엔진"""

    def __init__(self, config=None):
        self.cfg = config or INDICATOR_CONFIG

    def classify_rsi(self, rsi, close):
        val = rsi.iloc[-1]
        prev = rsi.iloc[-2] if len(rsi) > 1 else val
        if np.isnan(val):
            return self._na_signal('RSI', 'momentum')
        if val > self.cfg.rsi_overbought:
            sig, desc = 'buy', f'{val:.1f} 금리과매수→매수'
        elif val < self.cfg.rsi_oversold:
            sig, desc = 'sell', f'{val:.1f} 금리과매도→매도'
        else:
            sig, desc = 'neutral', f'{val:.1f} 중립'
        abs_gauge = 1.0 - val / 100.0
        return IndicatorSignal(
            name='RSI', category='momentum', value=val, signal=sig,
            description=desc, abs_gauge=abs_gauge,
            rel_gauges=self._percentile_gauges(rsi),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(rsi, close)
        )

    def classify_stochastic(self, k, d, close):
        val = k.iloc[-1]
        prev = k.iloc[-2] if len(k) > 1 else val
        if np.isnan(val):
            return self._na_signal('Stochastic', 'momentum')
        if val > self.cfg.stoch_overbought:
            sig, desc = 'buy', f'%K={val:.0f} 금리과매수'
        elif val < self.cfg.stoch_oversold:
            sig, desc = 'sell', f'%K={val:.0f} 금리과매도'
        else:
            sig, desc = 'neutral', f'%K={val:.0f} 중립'
        return IndicatorSignal(
            name='Stochastic', category='momentum', value=val, signal=sig,
            description=desc, abs_gauge=1.0 - val / 100.0,
            rel_gauges=self._percentile_gauges(k),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(k, close)
        )

    def classify_williams(self, wr, close):
        val = wr.iloc[-1]
        prev = wr.iloc[-2] if len(wr) > 1 else val
        if np.isnan(val):
            return self._na_signal('Williams %R', 'momentum')
        if val > self.cfg.williams_overbought:
            sig, desc = 'buy', f'{val:.0f} 금리과매수→매수'
        elif val < self.cfg.williams_oversold:
            sig, desc = 'sell', f'{val:.0f} 금리과매도→매도'
        else:
            sig, desc = 'neutral', f'{val:.0f} 중립'
        abs_gauge = (val + 100) / 100.0
        return IndicatorSignal(
            name='Williams %R', category='momentum', value=val, signal=sig,
            description=desc, abs_gauge=abs_gauge,
            rel_gauges=self._percentile_gauges(wr),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(wr, close)
        )

    def classify_macd(self, macd, signal, hist, close):
        val = hist.iloc[-1]
        prev = hist.iloc[-2] if len(hist) > 1 else val
        macd_val = macd.iloc[-1]
        if np.isnan(val):
            return self._na_signal('MACD', 'trend')
        if val < 0 and (len(hist) < 2 or hist.iloc[-2] >= 0):
            sig, desc = 'buy', f'{macd_val:.4f} 데드크로스'
        elif val > 0 and (len(hist) < 2 or hist.iloc[-2] <= 0):
            sig, desc = 'sell', f'{macd_val:.4f} 골든크로스'
        elif val < 0:
            sig, desc = 'buy', f'{macd_val:.4f} 하락추세'
        elif val > 0:
            sig, desc = 'sell', f'{macd_val:.4f} 상승추세'
        else:
            sig, desc = 'neutral', f'{macd_val:.4f} 중립'
        return IndicatorSignal(
            name='MACD', category='trend', value=macd_val, signal=sig,
            description=desc, abs_gauge=0.5,
            rel_gauges=self._percentile_gauges(hist),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(macd, close)
        )

    def classify_dmi(self, plus_di, minus_di, adx):
        pdi, mdi, adx_val = plus_di.iloc[-1], minus_di.iloc[-1], adx.iloc[-1]
        prev_adx = adx.iloc[-2] if len(adx) > 1 else adx_val
        if any(np.isnan(v) for v in [pdi, mdi, adx_val]):
            return self._na_signal('DMI', 'trend')
        if mdi > pdi and adx_val > 25:
            sig, desc = 'buy', f'ADX {adx_val:.0f} 금리하락추세'
        elif pdi > mdi and adx_val > 25:
            sig, desc = 'sell', f'ADX {adx_val:.0f} 금리상승추세'
        else:
            sig, desc = 'neutral', f'ADX {adx_val:.0f} {"추세 강화중" if adx_val > 20 else "비추세"}'
        abs_gauge = max(0, min(1, 0.5 - (pdi - mdi) / 100.0))
        return IndicatorSignal(
            name='DMI', category='trend', value=adx_val, signal=sig,
            description=desc, abs_gauge=abs_gauge,
            rel_gauges=self._percentile_gauges(adx),
            direction=self._direction(adx_val, prev_adx),
            divergence=False
        )

    def classify_cci(self, cci, close):
        val = cci.iloc[-1]
        prev = cci.iloc[-2] if len(cci) > 1 else val
        if np.isnan(val):
            return self._na_signal('CCI', 'trend')
        if val > self.cfg.cci_overbought:
            sig, desc = 'buy', f'{val:.0f} 금리과매수→매수'
        elif val < self.cfg.cci_oversold:
            sig, desc = 'sell', f'{val:.0f} 금리과매도→매도'
        else:
            sig, desc = 'neutral', f'{val:.0f} 중립'
        abs_gauge = max(0, min(1, (val + 300) / 600.0))
        return IndicatorSignal(
            name='CCI', category='trend', value=val, signal=sig,
            description=desc, abs_gauge=abs_gauge,
            rel_gauges=self._percentile_gauges(cci),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(cci, close)
        )

    def classify_bollinger(self, close, pctb, bw):
        val = pctb.iloc[-1]
        prev = pctb.iloc[-2] if len(pctb) > 1 else val
        if np.isnan(val):
            return self._na_signal('Bollinger', 'volatility')
        if val > 1.0:
            sig, desc = 'buy', f'%B={val:.2f} 상한돌파→매수'
        elif val < 0.0:
            sig, desc = 'sell', f'%B={val:.2f} 하한돌파→매도'
        elif val > 0.8:
            sig, desc = 'buy', f'%B={val:.2f} 상한근접'
        elif val < 0.2:
            sig, desc = 'sell', f'%B={val:.2f} 하한근접'
        else:
            sig, desc = 'neutral', f'%B={val:.2f} 중립'
        return IndicatorSignal(
            name='Bollinger', category='volatility', value=val, signal=sig,
            description=desc, abs_gauge=max(0, min(1, val)),
            rel_gauges=self._percentile_gauges(pctb),
            direction=self._direction(val, prev),
            divergence=False
        )

    def classify_atr(self, atr):
        val = atr.iloc[-1]
        prev = atr.iloc[-2] if len(atr) > 1 else val
        if np.isnan(val):
            return self._na_signal('ATR', 'volatility')
        return IndicatorSignal(
            name='ATR', category='volatility', value=val, signal='neutral',
            description=f'{val:.4f} {"고변동" if val > atr.quantile(0.75) else "보통"}',
            abs_gauge=0.5,
            rel_gauges=self._percentile_gauges(atr),
            direction=self._direction(val, prev),
            divergence=False
        )

    def classify_obv(self, obv, close):
        val = obv.iloc[-1]
        prev = obv.iloc[-2] if len(obv) > 1 else val
        if np.isnan(val):
            return self._na_signal('OBV', 'volume')
        obv_ma = obv.rolling(20).mean()
        obv_trend = 'rising' if val > obv_ma.iloc[-1] else 'falling'
        if obv_trend == 'rising':
            sig, desc = 'sell', '상승 추세'
        else:
            sig, desc = 'buy', '하락 추세'
        return IndicatorSignal(
            name='OBV', category='volume', value=val, signal=sig,
            description=desc, abs_gauge=0.5,
            rel_gauges=self._percentile_gauges(obv),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(obv, close)
        )

    def classify_cmf(self, cmf):
        val = cmf.iloc[-1]
        prev = cmf.iloc[-2] if len(cmf) > 1 else val
        if np.isnan(val):
            return self._na_signal('CMF', 'volume')
        if val > 0.1:
            sig, desc = 'sell', f'+{val:.2f} 강한 유입'
        elif val > 0:
            sig, desc = 'neutral', f'+{val:.2f} 약한 유입'
        elif val > -0.1:
            sig, desc = 'neutral', f'{val:.2f} 약한 유출'
        else:
            sig, desc = 'buy', f'{val:.2f} 강한 유출'
        abs_gauge = max(0, min(1, 0.5 - val / 2.0))
        return IndicatorSignal(
            name='CMF', category='volume', value=val, signal=sig,
            description=desc, abs_gauge=abs_gauge,
            rel_gauges=self._percentile_gauges(cmf),
            direction=self._direction(val, prev),
            divergence=False
        )

    def classify_sonar(self, sonar, close):
        val = sonar.iloc[-1]
        prev = sonar.iloc[-2] if len(sonar) > 1 else val
        if np.isnan(val):
            return self._na_signal('SONAR', 'trend')
        if val < 0:
            sig, desc = 'buy', f'{val:.2f} 하락모멘텀→매수'
        elif val > 0:
            sig, desc = 'sell', f'{val:.2f} 상승모멘텀→매도'
        else:
            sig, desc = 'neutral', f'{val:.2f} 중립'
        return IndicatorSignal(
            name='SONAR', category='trend', value=val, signal=sig,
            description=desc, abs_gauge=0.5,
            rel_gauges=self._percentile_gauges(sonar),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(sonar, close)
        )

    def classify_chaikin_vol(self, chvol):
        val = chvol.iloc[-1]
        prev = chvol.iloc[-2] if len(chvol) > 1 else val
        if np.isnan(val):
            return self._na_signal('Chaikin Vol', 'volatility')
        if val > 0.1:
            desc = f'{val:.3f} 변동성 확대'
        elif val < -0.1:
            desc = f'{val:.3f} 변동성 축소'
        else:
            desc = f'{val:.3f} 보통'
        return IndicatorSignal(
            name='Chaikin Vol', category='volatility', value=val, signal='neutral',
            description=desc, abs_gauge=0.5,
            rel_gauges=self._percentile_gauges(chvol),
            direction=self._direction(val, prev),
            divergence=False
        )

    def classify_ad_line(self, ad, close):
        val = ad.iloc[-1]
        prev = ad.iloc[-2] if len(ad) > 1 else val
        if np.isnan(val):
            return self._na_signal('A/D Line', 'volume')
        ad_ma = ad.rolling(20).mean()
        ad_trend = 'rising' if val > ad_ma.iloc[-1] else 'falling'
        if ad_trend == 'rising':
            sig, desc = 'sell', '매집→금리상승압력'
        else:
            sig, desc = 'buy', '분산→금리하락압력'
        return IndicatorSignal(
            name='A/D Line', category='volume', value=val, signal=sig,
            description=desc, abs_gauge=0.5,
            rel_gauges=self._percentile_gauges(ad),
            direction=self._direction(val, prev),
            divergence=self._check_divergence(ad, close)
        )

    def _na_signal(self, name, category):
        return IndicatorSignal(
            name=name, category=category, value=np.nan,
            signal='n/a', description='데이터 부족', abs_gauge=0.5,
            rel_gauges={'full': 0.5, 'short': 0.5, 'medium': 0.5, 'long': 0.5},
            direction='stable', divergence=False
        )

    def _percentile_gauges(self, series):
        val = series.iloc[-1]
        if np.isnan(val):
            return {'full': 0.5, 'short': 0.5, 'medium': 0.5, 'long': 0.5}

        def _pct(subset):
            s = subset.dropna()
            if len(s) < 5:
                return 0.5
            return stats.percentileofscore(s.values, val, kind='rank') / 100.0

        result = {}
        result['full'] = _pct(series)
        for label, window in [('short', self.cfg.gauge_short),
                               ('medium', self.cfg.gauge_medium),
                               ('long', self.cfg.gauge_long)]:
            result[label] = _pct(series.iloc[-window:])
        return result

    def _direction(self, current, previous):
        if np.isnan(current) or np.isnan(previous):
            return 'stable'
        diff = current - previous
        if abs(diff) < 1e-8:
            return 'stable'
        return 'strengthening' if diff > 0 else 'weakening'

    def _check_divergence(self, indicator, price, lookback=20):
        if len(indicator) < lookback or len(price) < lookback:
            return False
        ind_tail = indicator.iloc[-lookback:]
        price_tail = price.iloc[-lookback:]
        if ind_tail.isna().all() or price_tail.isna().all():
            return False
        ind_trend = np.polyfit(range(len(ind_tail.dropna())), ind_tail.dropna(), 1)[0]
        price_trend = np.polyfit(range(len(price_tail.dropna())), price_tail.dropna(), 1)[0]
        return (ind_trend > 0 and price_trend < 0) or (ind_trend < 0 and price_trend > 0)

    def classify_all(self, indicators, close):
        signals = []
        if 'RSI' in indicators:
            signals.append(self.classify_rsi(indicators['RSI'], close))
        if 'Stoch_K' in indicators and 'Stoch_D' in indicators:
            signals.append(self.classify_stochastic(indicators['Stoch_K'], indicators['Stoch_D'], close))
        if 'Williams_R' in indicators:
            signals.append(self.classify_williams(indicators['Williams_R'], close))
        if all(k in indicators for k in ['MACD', 'MACD_signal', 'MACD_hist']):
            signals.append(self.classify_macd(indicators['MACD'], indicators['MACD_signal'],
                                              indicators['MACD_hist'], close))
        if all(k in indicators for k in ['DMI_plus_di', 'DMI_minus_di', 'DMI_adx']):
            signals.append(self.classify_dmi(indicators['DMI_plus_di'], indicators['DMI_minus_di'],
                                             indicators['DMI_adx']))
        if 'CCI' in indicators:
            signals.append(self.classify_cci(indicators['CCI'], close))
        if 'BB_pctb' in indicators and 'BB_bandwidth' in indicators:
            signals.append(self.classify_bollinger(close, indicators['BB_pctb'], indicators['BB_bandwidth']))
        if 'ATR' in indicators:
            signals.append(self.classify_atr(indicators['ATR']))
        if 'OBV' in indicators:
            signals.append(self.classify_obv(indicators['OBV'], close))
        if 'CMF' in indicators:
            signals.append(self.classify_cmf(indicators['CMF']))
        if 'SONAR' in indicators:
            signals.append(self.classify_sonar(indicators['SONAR'], close))
        if 'Chaikin_Vol' in indicators:
            signals.append(self.classify_chaikin_vol(indicators['Chaikin_Vol']))
        if 'AD_Line' in indicators:
            signals.append(self.classify_ad_line(indicators['AD_Line'], close))
        return signals

    def composite_score(self, signals):
        valid = [s for s in signals if s.signal != 'n/a']
        if not valid:
            return {'score': 0, 'buy': 0, 'sell': 0, 'neutral': 0, 'na': len(signals)}
        buy_count = sum(1 for s in valid if s.signal == 'buy')
        sell_count = sum(1 for s in valid if s.signal == 'sell')
        neutral_count = sum(1 for s in valid if s.signal == 'neutral')
        na_count = sum(1 for s in signals if s.signal == 'n/a')
        score = (buy_count - sell_count) / len(valid) if valid else 0
        return {
            'score': round(score, 2), 'buy': buy_count, 'sell': sell_count,
            'neutral': neutral_count, 'na': na_count,
            'total': len(signals), 'valid': len(valid),
        }


# ============================================================================
# 전역 인스턴스
# ============================================================================

price_trend_engine = PriceTrendEngine()
vol_momentum_engine = VolatilityMomentumEngine()
volume_engine = VolumeStrengthEngine()
chart_renderer = SpecialChartRenderer()
indicator_overlay = IndicatorOverlay()
composite_dashboard = CompositeDashboard()
signal_classifier = SignalClassifier()

print("Cell 1 완료: 모듈 로딩 (유틸리티 + 엔진 + 렌더러 + 시그널)")


In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 2: 데이터 로딩
# ══════════════════════════════════════════════════════════════

import os

# 확장 로더 인스턴스 생성
bond_ohlcv_loader = BondOHLCVLoader()
futures_loader = FuturesLoader()

# 파일 경로 결정
_nb_dir = Path(os.getcwd())
_xlsb_path = _nb_dir / DATA_CONFIG.file_path

if not _xlsb_path.exists():
    raise FileNotFoundError(
        f"데이터 파일을 찾을 수 없습니다: {_xlsb_path}\n"
        f"TA_rawdata.xlsb 파일이 노트북과 같은 디렉토리에 있어야 합니다."
    )

print(f"데이터 파일: {_xlsb_path.name}")

# ── 1. 금리 데이터 (Rates 시트) ──
rates = read_rates_dataframe(
    _xlsb_path,
    DATA_CONFIG.rates_sheet,
    header_row=DATA_CONFIG.header_row,
    data_start_row=DATA_CONFIG.data_start_row
)

# ── 2. 스프레드 데이터 (Spread 시트) ──
spreads_vs_ktb = read_rates_dataframe(
    _xlsb_path,
    DATA_CONFIG.spread_sheet,
    header_row=DATA_CONFIG.header_row,
    data_start_row=DATA_CONFIG.data_start_row
)

# ── 3. 국고지표 OHLCV ──
bond_ohlcv = bond_ohlcv_loader.load_from_xlsb(str(_xlsb_path))

# ── 4. 국고선물 OHLCV ──
futures = futures_loader.load_from_xlsb(str(_xlsb_path))

# ── 5. 데이터 티어 탐지기 ──
tier_detector = DataTierDetector(bond_ohlcv_loader, futures_loader)

# ── 결과 요약 ──
print(f"\n{'='*60}")
print(f"데이터 로딩 완료")
print(f"{'='*60}")

if not rates.empty:
    print(f"금리 데이터: {rates.shape}")
    print(f"스프레드 데이터: {spreads_vs_ktb.shape}")
    print(f"기간: {rates.index[0].date()} ~ {rates.index[-1].date()}")
    print(f"거래일: {len(rates)}일")
    print(f"종목 수: {len(rates.columns)}개")
else:
    print("금리/스프레드: 데이터 없음")

if bond_ohlcv:
    print(f"OHLCV 종목: {list(bond_ohlcv.keys())}")
if futures:
    print(f"선물 종목: {list(futures.keys())}")

if bond_ohlcv or futures:
    print(f"\n[데이터 티어 요약]")
    print(tier_detector.summary().to_string(index=False))

# 경고 억제
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
logging.getLogger('matplotlib.category').setLevel(logging.ERROR)

# 저장 디렉토리 생성
if CFG.save_dir and (CFG.save_excel or CFG.save_graphs):
    CFG.save_dir.mkdir(parents=True, exist_ok=True)
    print(f"\n저장 디렉토리: {CFG.save_dir.resolve()}")

print("\nCell 2 완료: 데이터 로딩")


In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 3: 상관분석 (히트맵 + 롤링상관 + 금리비교)
# ══════════════════════════════════════════════════════════════

from itertools import combinations
from matplotlib.colors import TwoSlopeNorm

# ── 함수 정의 ──

def build_spreads_corr(rates_df, columns):
    """종목 쌍별 스프레드 계산"""
    result = pd.DataFrame(index=rates_df.index)
    for i, j in combinations(range(len(columns)), 2):
        name = f"{columns[j]} - {columns[i]}"
        result[name] = rates_df[columns[j]] - rates_df[columns[i]]
    return result


def build_analysis_data(rates_df, spreads_df, columns, mode):
    """모드에 따른 분석 데이터 생성"""
    source = spreads_df if mode in ['spreads_vs_ktb', 'spreads_vs_ktb_changes'] else rates_df
    if columns is None:
        cols = source.columns.tolist()
    else:
        cols = [c for c in columns if c in source.columns]
        if not cols:
            raise ValueError("유효한 종목이 없습니다")
    if mode == 'rates': return source[cols].dropna()
    elif mode == 'rates_changes': return source[cols].diff().dropna()
    elif mode == 'spreads': return build_spreads_corr(source, cols).dropna()
    elif mode == 'spreads_changes': return build_spreads_corr(source, cols).diff().dropna()
    elif mode == 'spreads_vs_ktb': return source[cols].dropna()
    elif mode == 'spreads_vs_ktb_changes': return source[cols].diff().dropna()
    else: raise ValueError(f"알 수 없는 모드: {mode}")


def plot_correlation_heatmap(data, **kwargs):
    """상관관계 히트맵 (A4 Landscape)"""
    start_date = kwargs.get('start_date')
    end_date = kwargs.get('end_date')
    mode = kwargs.get('mode', '')

    df = data.copy()
    if start_date: df = df[df.index >= pd.to_datetime(start_date)]
    if end_date: df = df[df.index <= pd.to_datetime(end_date)]
    if df.empty: raise ValueError("데이터가 없습니다")

    corr = df.corr()
    n = len(corr)
    disp_arr = corr.to_numpy(dtype=float, copy=True)
    np.fill_diagonal(disp_arr, np.nan)
    display = pd.DataFrame(disp_arr, index=corr.index, columns=corr.columns)

    fig, ax = plt.subplots(figsize=A4_LANDSCAPE)
    im = ax.imshow(display.values, cmap='RdYlBu_r',
                   norm=TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1), aspect='equal')
    plt.colorbar(im, ax=ax, shrink=0.8, label='Correlation')

    for i in range(n):
        for j in range(n):
            val = display.iloc[i, j]
            if not np.isnan(val):
                color = 'white' if abs(val) > 0.6 else 'black'
                ax.text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=8, color=color)

    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels(corr.columns, fontsize=CHART_STYLE['label_fontsize'], rotation=45, ha='right')
    ax.set_yticklabels(corr.index, fontsize=CHART_STYLE['label_fontsize'])
    ax.set_xticks(np.arange(-0.5, n, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, n, 1), minor=True)
    ax.grid(which='minor', color='white', linestyle='-', linewidth=1)
    ax.tick_params(which='minor', size=0)

    mode_names = {
        'rates': '금리 수준', 'rates_changes': '금리 변화',
        'spreads': '종목 간 스프레드', 'spreads_changes': '종목 간 스프레드 변화',
        'spreads_vs_ktb': '국고채 대비 스프레드', 'spreads_vs_ktb_changes': '국고채 대비 스프레드 변화'
    }
    start_str = df.index[0].strftime('%Y-%m-%d')
    end_str = df.index[-1].strftime('%Y-%m-%d')
    ax.set_title(f"상관관계 히트맵 ({mode_names.get(mode, mode)})\n{start_str} ~ {end_str}, {len(df)}일",
                 fontsize=CHART_STYLE['title_fontsize'], fontweight='bold', pad=20)
    plt.tight_layout()
    _all_figs.append((fig, 'heatmap'))
    plt.show()
    return corr


def calculate_rolling_corr(data, col1, col2, window):
    return data[col1].rolling(window=window).corr(data[col2])


def calculate_rolling_stats(series, window):
    rolling_mean = series.rolling(window=window).mean()
    rolling_std = series.rolling(window=window).std()
    z_score = (series - rolling_mean) / rolling_std
    return rolling_mean, rolling_std, z_score


def plot_rolling_analysis(data_full, data_filtered, data_hist, col1, col2,
                          pair_name, window, hist_period_label):
    """단일 쌍 롤링 분석 그래프 (A4 Landscape)"""
    corr_hist = calculate_rolling_corr(data_hist, col1, col2, window).dropna()
    corr_filtered = calculate_rolling_corr(data_filtered, col1, col2, window).dropna()
    corr_mean = corr_filtered.rolling(window=window).mean()
    corr_std = corr_filtered.rolling(window=window).std()

    spread_hist = data_hist[col2] - data_hist[col1]
    spread_filtered = data_filtered[col2] - data_filtered[col1]
    spread_mean, spread_std, spread_zscore = calculate_rolling_stats(spread_filtered, window)
    _, spread_std_hist, spread_zscore_hist = calculate_rolling_stats(spread_hist, window)

    corr_pct = stats.percentileofscore(corr_hist.dropna(), corr_filtered.iloc[-1])
    spread_pct = stats.percentileofscore(spread_hist.dropna(), spread_filtered.iloc[-1])

    fig = plt.figure(figsize=A4_LANDSCAPE, constrained_layout=True)
    gs = fig.add_gridspec(4, 6, height_ratios=[1.2, 1, 1, 1],
                          width_ratios=[1, 1, 1, 1, 1, 0.7], hspace=0.08, wspace=0.08)

    ax_corr = fig.add_subplot(gs[0, :5])
    ax_corr_hist = fig.add_subplot(gs[0, 5])
    ax_spread = fig.add_subplot(gs[1, :5], sharex=ax_corr)
    ax_spread_hist = fig.add_subplot(gs[1, 5])
    ax_std = fig.add_subplot(gs[2, :5], sharex=ax_corr)
    ax_std_hist = fig.add_subplot(gs[2, 5])
    ax_zscore = fig.add_subplot(gs[3, :5], sharex=ax_corr)
    ax_zscore_hist = fig.add_subplot(gs[3, 5])

    lw = CHART_STYLE['linewidth']

    # 1. 롤링 상관계수
    upper_2s = (corr_mean + 2 * corr_std).clip(-1, 1)
    lower_2s = (corr_mean - 2 * corr_std).clip(-1, 1)
    ax_corr.fill_between(corr_filtered.index, lower_2s, upper_2s, color='#e8e8e8', alpha=0.7)
    upper_1s = (corr_mean + corr_std).clip(-1, 1)
    lower_1s = (corr_mean - corr_std).clip(-1, 1)
    ax_corr.plot(corr_filtered.index, upper_1s, '--', color='red', linewidth=0.5, alpha=0.7)
    ax_corr.plot(corr_filtered.index, lower_1s, '--', color='blue', linewidth=0.5, alpha=0.7)
    ax_corr.plot(corr_filtered.index, corr_filtered, color='black', linewidth=lw)
    ax_corr.axhline(y=0, color='gray', linewidth=0.5)
    ax_corr.set_ylabel('Corr', fontsize=CHART_STYLE['label_fontsize'])
    ax_corr.set_ylim(-1.05, 1.05)
    ax_corr.set_title(f'{pair_name}  (Rolling {window}일)', fontsize=CHART_STYLE['title_fontsize'], fontweight='bold')
    setup_date_axis(ax_corr, corr_filtered.index)
    ax_corr.tick_params(axis='x', labelbottom=False)

    ax_corr_hist.hist(corr_hist.dropna(), bins=30, orientation='horizontal', color='black', alpha=0.4, edgecolor='white')
    ax_corr_hist.axhline(y=corr_filtered.iloc[-1], color='red', linewidth=1.5,
                         label=f'{corr_filtered.iloc[-1]:.2f} ({corr_pct:.0f}%ile)')
    ax_corr_hist.set_title(f'분포 ({hist_period_label})', fontsize=7)
    ax_corr_hist.legend(loc='upper right', fontsize=5)
    ax_corr_hist.yaxis.tick_right()
    ax_corr_hist.set_ylim(-1.05, 1.05)

    # 2. 스프레드
    ax_spread.plot(spread_filtered.index, spread_filtered, color='#1f77b4', linewidth=lw)
    ax_spread.plot(spread_mean.index, spread_mean, color='#ff7f0e', linewidth=0.8)
    ax_spread.axhline(y=0, color='gray', linewidth=0.5)
    ax_spread.set_ylabel('Spread', fontsize=CHART_STYLE['label_fontsize'])
    ax_spread.tick_params(axis='x', labelbottom=False)

    ax_spread_hist.hist(spread_hist.dropna(), bins=30, orientation='horizontal', color='#1f77b4', alpha=0.4, edgecolor='white')
    ax_spread_hist.axhline(y=spread_filtered.iloc[-1], color='red', linewidth=1.5,
                           label=f'{spread_filtered.iloc[-1]:.2f} ({spread_pct:.0f}%ile)')
    ax_spread_hist.legend(loc='upper right', fontsize=5)
    ax_spread_hist.yaxis.tick_right()

    # 3. 변동성
    ax_std.plot(spread_std.index, spread_std, color='#2ca02c', linewidth=lw)
    ax_std.set_ylabel('Vol', fontsize=CHART_STYLE['label_fontsize'])
    ax_std.tick_params(axis='x', labelbottom=False)

    std_pct = stats.percentileofscore(spread_std_hist.dropna(), spread_std.iloc[-1])
    ax_std_hist.hist(spread_std_hist.dropna(), bins=30, orientation='horizontal', color='#2ca02c', alpha=0.4, edgecolor='white')
    ax_std_hist.axhline(y=spread_std.iloc[-1], color='red', linewidth=1.5,
                        label=f'{spread_std.iloc[-1]:.4f} ({std_pct:.0f}%ile)')
    ax_std_hist.legend(loc='upper right', fontsize=5)
    ax_std_hist.yaxis.tick_right()

    # 4. Z-score
    ax_zscore.fill_between(spread_zscore.index, -2, 2, color='#e8e8e8', alpha=0.7)
    ax_zscore.axhline(y=1, color='black', linewidth=0.5, linestyle='--', alpha=0.7)
    ax_zscore.axhline(y=-1, color='black', linewidth=0.5, linestyle='--', alpha=0.7)
    ax_zscore.plot(spread_zscore.index, spread_zscore, color='#ff7f0e', linewidth=lw)
    ax_zscore.axhline(y=0, color='gray', linewidth=0.5)
    ax_zscore.set_ylabel('Z-score', fontsize=CHART_STYLE['label_fontsize'])
    ax_zscore.set_ylim(-4, 4)
    setup_date_axis(ax_zscore, spread_zscore.index)

    zscore_pct = stats.percentileofscore(spread_zscore_hist.dropna(), spread_zscore.iloc[-1])
    ax_zscore_hist.hist(spread_zscore_hist.dropna(), bins=30, orientation='horizontal', color='#ff7f0e', alpha=0.4, edgecolor='white')
    ax_zscore_hist.axhline(y=spread_zscore.iloc[-1], color='red', linewidth=1.5,
                           label=f'{spread_zscore.iloc[-1]:.2f}σ ({zscore_pct:.0f}%ile)')
    ax_zscore_hist.legend(loc='upper right', fontsize=5)
    ax_zscore_hist.yaxis.tick_right()
    ax_zscore_hist.set_ylim(-4, 4)

    _all_figs.append((fig, 'rolling'))
    plt.show()

    print(f"  {pair_name}: corr={corr_filtered.iloc[-1]:.3f}({corr_pct:.0f}%ile) spread={spread_filtered.iloc[-1]:.4f} z={spread_zscore.iloc[-1]:.2f}σ")
    return {'pair_name': pair_name, 'fig': fig}


def plot_rate_comparison(data, returns, pairs, window, start_date, end_date):
    """금리 수준 비교 (A4 Landscape)"""
    plot_data = data.copy()
    plot_returns = returns.copy()
    if start_date:
        plot_data = plot_data[plot_data.index >= pd.to_datetime(start_date)]
        plot_returns = plot_returns[plot_returns.index >= pd.to_datetime(start_date)]
    if end_date:
        plot_data = plot_data[plot_data.index <= pd.to_datetime(end_date)]
        plot_returns = plot_returns[plot_returns.index <= pd.to_datetime(end_date)]

    n_total = len(pairs)
    if n_total == 0: return
    period_label = f"{plot_data.index[0].strftime('%Y-%m-%d')} ~ {plot_data.index[-1].strftime('%Y-%m-%d')}"

    MAX_PER_PAGE = 12
    n_pages = (n_total + MAX_PER_PAGE - 1) // MAX_PER_PAGE

    for page in range(n_pages):
        si = page * MAX_PER_PAGE
        page_pairs = pairs[si:si + MAX_PER_PAGE]
        n_items = len(page_pairs)

        if n_items <= 2: nrows, ncols = 1, max(1, n_items)
        elif n_items <= 4: nrows, ncols = 2, 2
        elif n_items <= 6: nrows, ncols = 2, 3
        elif n_items <= 9: nrows, ncols = 3, 3
        else: nrows, ncols = 3, 4

        fig, axes = plt.subplots(nrows, ncols, figsize=A4_LANDSCAPE, constrained_layout=True)
        if nrows == 1 and ncols == 1: axes = np.array([[axes]])
        elif nrows == 1: axes = axes.reshape(1, -1)
        elif ncols == 1: axes = axes.reshape(-1, 1)

        page_label = f" ({page+1}/{n_pages})" if n_pages > 1 else ""
        fig.suptitle(f'금리 수준 비교{page_label}\n{period_label}', fontsize=10, fontweight='bold')

        for idx, (display_name, black_col, blue_col) in enumerate(page_pairs):
            ax = axes[idx // ncols, idx % ncols]
            ax.plot(plot_data.index, plot_data[black_col], color='black', linewidth=CHART_STYLE['linewidth'],
                    label=black_col.replace('RF_', '_'))
            ax.plot(plot_data.index, plot_data[blue_col], color=CHART_STYLE['color_secondary'],
                    linewidth=CHART_STYLE['linewidth'], label=blue_col.replace('RF_', '_'))

            corr = plot_returns[black_col].corr(plot_returns[blue_col]) if (
                black_col in plot_returns.columns and blue_col in plot_returns.columns) else np.nan
            ax.set_title(f"{black_col.replace('RF_', '_')} vs {blue_col.replace('RF_', '_')}\nr={corr:.3f}",
                         fontsize=7)
            setup_date_axis(ax, plot_data.index)
            apply_chart_style(ax)
            ax.legend(loc='best', fontsize=5)

        for idx in range(n_items, nrows * ncols):
            axes[idx // ncols, idx % ncols].axis('off')

        _all_figs.append((fig, 'comparison'))
        plt.show()


def _get_compare_pairs(columns):
    """비교 쌍 자동 생성 (만기 > 등급 > 섹터 순서)"""
    pairs = []
    for c1, c2 in combinations(columns, 2):
        pairs.append((f"{c1} vs {c2}", c1, c2))
    return pairs


# ── 실행 ──

_all_figs = []  # (type, fig) 리스트 — Cell 6에서 저장용

print(f"{'='*60}")
print("Cell 3: 상관분석")
print(f"{'='*60}")

# (1) 히트맵
print(f"\n[1/3] 히트맵: {CFG.corr_columns}, 모드={CFG.corr_mode}")
analysis_data = build_analysis_data(rates, spreads_vs_ktb, CFG.corr_columns, CFG.corr_mode)
corr_matrix = plot_correlation_heatmap(
    analysis_data, start_date=CFG.start_date, end_date=CFG.end_date, mode=CFG.corr_mode)

# (2) 롤링 상관계수
print(f"\n[2/3] 롤링 상관계수: 윈도우={CFG.rolling_window}일")

_corr_cols = CFG.corr_columns
_corr_mode = CFG.corr_mode
_data_full = build_analysis_data(rates, spreads_vs_ktb, _corr_cols, _corr_mode)

_data_filtered = _data_full.copy()
if CFG.start_date:
    _data_filtered = _data_filtered[_data_filtered.index >= pd.to_datetime(CFG.start_date)]
if CFG.end_date:
    _data_filtered = _data_filtered[_data_filtered.index <= pd.to_datetime(CFG.end_date)]

_data_hist = _data_full.copy()  # 히스토그램 기준 = 전체 기간

_hist_label = f"{_data_hist.index[0].strftime('%Y.%m')}~{_data_hist.index[-1].strftime('%Y.%m')}"

if _corr_mode in ['spreads', 'spreads_changes']:
    _spread_names = []
    for i, j in combinations(range(len(_corr_cols)), 2):
        _spread_names.append(f"{_corr_cols[j]} - {_corr_cols[i]}")
    _rolling_pairs = [(f"({_spread_names[i]}) vs ({_spread_names[j]})", _spread_names[i], _spread_names[j])
                      for i, j in combinations(range(len(_spread_names)), 2)]
else:
    _rolling_pairs = [(f"{_corr_cols[i]} vs {_corr_cols[j]}", _corr_cols[i], _corr_cols[j])
                      for i, j in combinations(range(len(_corr_cols)), 2)]

for pair_name, col1, col2 in _rolling_pairs:
    plot_rolling_analysis(_data_full, _data_filtered, _data_hist, col1, col2,
                          pair_name, CFG.rolling_window, _hist_label)

# (3) 금리 수준 비교
print(f"\n[3/3] 금리 수준 비교: {len(CFG.compare_columns)}종목")
_compare_pairs = _get_compare_pairs(CFG.compare_columns)
_compare_returns = rates[CFG.compare_columns].diff().dropna()
plot_rate_comparison(rates, _compare_returns, _compare_pairs, CFG.rolling_window,
                     CFG.compare_start, CFG.compare_end)

print(f"\nCell 3 완료: 상관분석 ({len(_all_figs)}개 차트)")


In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 4: 기술적 지표 대시보드
# ══════════════════════════════════════════════════════════════

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import pandas as pd
import numpy as np

# ── Step 1: 데이터 준비 ──────────────────────────────────────

tier = tier_detector.detect(CFG.target)
is_yield = CFG.data_source in ('bond_close', 'bond_ohlcv', 'futures_yield')

if CFG.data_source == 'bond_close':
    col_name = [c for c in rates.columns
                if CFG.target.replace(' ', '') in c.replace(' ', '')]
    if not col_name:
        raise ValueError(f"종목 '{CFG.target}' not found in rates")
    close_series = rates[col_name[0]].dropna()
    ohlcv_df = pd.DataFrame({'close': close_series})
    current_tier = '2'
elif CFG.data_source == 'bond_ohlcv':
    ohlcv_df = tier_detector.get_ohlcv(CFG.target, source_preference='bond_ohlcv')
    if ohlcv_df is None:
        raise ValueError(f"종목 '{CFG.target}'에 대한 OHLCV 데이터 없음")
    current_tier = '1b'
elif CFG.data_source == 'futures_price':
    ohlcv_df = tier_detector.get_ohlcv(CFG.target, mode='price')
    if ohlcv_df is None:
        raise ValueError(f"종목 '{CFG.target}'에 대한 선물 가격 데이터 없음")
    current_tier = tier.tier
elif CFG.data_source == 'futures_yield':
    ohlcv_df = tier_detector.get_ohlcv(CFG.target, mode='yield')
    if ohlcv_df is None:
        raise ValueError(f"종목 '{CFG.target}'에 대한 선물 수익률 데이터 없음")
    current_tier = tier.tier
else:
    raise ValueError(f"알 수 없는 data_source: {CFG.data_source}")

# 기간 필터
if CFG.start_date:
    ohlcv_df = ohlcv_df[ohlcv_df.index >= pd.to_datetime(CFG.start_date)]
if CFG.end_date:
    ohlcv_df = ohlcv_df[ohlcv_df.index <= pd.to_datetime(CFG.end_date)]

if ohlcv_df.empty:
    raise ValueError("필터링 후 데이터가 없습니다")

# 지표 계산
indicators = {}
indicators.update(price_trend_engine.compute_all(ohlcv_df, current_tier))
indicators.update(vol_momentum_engine.compute_all(ohlcv_df, current_tier))
indicators.update(volume_engine.compute_all(ohlcv_df, current_tier))

# 편의 변수
close = ohlcv_df['close']
dates = ohlcv_df.index
has_ohlc = all(c in ohlcv_df.columns for c in ['open', 'high', 'low', 'close'])
has_volume = ('volume' in ohlcv_df.columns
              and ohlcv_df['volume'].notna().any())

# MA 색상
MA_COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# ── Step 2: 결과 수집 리스트 ─────────────────────────────────
_all_figs = []


# ── 유틸리티 ─────────────────────────────────────────────────

def _style_ax(ax):
    """apply_chart_style + 스파인 제거"""
    apply_chart_style(ax)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)


def _finish_figure(fig, name='chart', tight_pad=1.5):
    """tight_layout 적용 후 _all_figs에 (fig, name) 튜플로 추가"""
    fig.tight_layout(pad=tight_pad)
    _all_figs.append((fig, name))


# ══════════════════════════════════════════════════════════════
# Page 1: 가격 + 추세 (A4 Portrait)
# ══════════════════════════════════════════════════════════════

_p1_ratios = [4, 2, 2, 1, 1]
fig1, axes1 = plt.subplots(
    len(_p1_ratios), 1,
    figsize=A4_PORTRAIT,
    height_ratios=_p1_ratios,
    sharex=True,
)

# ── Panel 1: Close + MA + Bollinger Bands ──
ax = axes1[0]
ax.plot(dates, close, color='black', linewidth=CHART_STYLE['linewidth'],
        label='Close', zorder=3)

# SMA overlays
_sma_periods = [5, 20, 60, 120]
for i, p in enumerate(_sma_periods):
    key = f'SMA_{p}'
    if key in indicators:
        ax.plot(dates, indicators[key],
                color=MA_COLORS[i % len(MA_COLORS)],
                linewidth=CHART_STYLE['linewidth'],
                label=key, alpha=0.8)

# Bollinger Bands
if 'BB_upper' in indicators:
    ax.plot(dates, indicators['BB_upper'], color='gray',
            linewidth=0.5, linestyle='--', alpha=0.6)
    ax.plot(dates, indicators['BB_lower'], color='gray',
            linewidth=0.5, linestyle='--', alpha=0.6)
    ax.fill_between(dates, indicators['BB_upper'], indicators['BB_lower'],
                    alpha=0.06, color='gray')

ax.set_ylabel('수익률 (%)' if is_yield else '가격',
              fontsize=CHART_STYLE['label_fontsize'])
ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'],
          ncol=3, framealpha=0.7)
ax.set_title(f'{CFG.target} — 가격 + 추세 지표 ({CFG.data_source})',
             fontsize=CHART_STYLE['title_fontsize'], fontweight='bold')
_style_ax(ax)

# ── Panel 2: MACD ──
ax = axes1[1]
if 'MACD' in indicators:
    ax.plot(dates, indicators['MACD'], color='#1f77b4',
            linewidth=CHART_STYLE['linewidth'], label='MACD')
    ax.plot(dates, indicators['MACD_signal'], color='#ff7f0e',
            linewidth=CHART_STYLE['linewidth'], label='Signal')
    hist = indicators['MACD_hist']
    colors_hist = ['#2ca02c' if v >= 0 else '#d62728'
                   for v in hist.values]
    ax.bar(dates, hist, color=colors_hist, width=0.8, alpha=0.6)
    ax.axhline(0, color='black', linewidth=0.3)
    ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'])
ax.set_ylabel('MACD', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 3: DMI (+DI, -DI, ADX) ──
ax = axes1[2]
if 'DMI_plus_di' in indicators:
    ax.plot(dates, indicators['DMI_plus_di'], color='#2ca02c',
            linewidth=CHART_STYLE['linewidth'], label='+DI')
    ax.plot(dates, indicators['DMI_minus_di'], color='#d62728',
            linewidth=CHART_STYLE['linewidth'], label='-DI')
    ax.plot(dates, indicators['DMI_adx'], color='black',
            linewidth=CHART_STYLE['linewidth'], label='ADX')
    ax.axhline(25, color='gray', linewidth=0.4, linestyle='--', alpha=0.5)
    ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'])
elif current_tier == '2':
    ax.text(0.5, 0.5, 'DMI: OHLC 데이터 필요 (Tier 2 미지원)',
            transform=ax.transAxes, ha='center', va='center',
            fontsize=CHART_STYLE['label_fontsize'], color='gray')
ax.set_ylabel('DMI', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 4: CCI ──
ax = axes1[3]
if 'CCI' in indicators:
    ax.plot(dates, indicators['CCI'], color='#9467bd',
            linewidth=CHART_STYLE['linewidth'])
    ax.axhline(100, color='red', linewidth=0.4, linestyle='--', alpha=0.6)
    ax.axhline(-100, color='green', linewidth=0.4, linestyle='--', alpha=0.6)
    ax.axhline(0, color='black', linewidth=0.3)
elif current_tier == '2':
    ax.text(0.5, 0.5, 'CCI: OHLC 데이터 필요',
            transform=ax.transAxes, ha='center', va='center',
            fontsize=CHART_STYLE['label_fontsize'], color='gray')
ax.set_ylabel('CCI', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 5: SONAR ──
ax = axes1[4]
if 'SONAR' in indicators:
    ax.plot(dates, indicators['SONAR'], color='#8c564b',
            linewidth=CHART_STYLE['linewidth'])
    ax.axhline(0, color='black', linewidth=0.3)
ax.set_ylabel('SONAR', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)
setup_date_axis(ax, dates)

_finish_figure(fig1, 'price_trend')
plt.show()


# ══════════════════════════════════════════════════════════════
# Page 2: 모멘텀 + 과매수/과매도 (A4 Portrait)
# ══════════════════════════════════════════════════════════════

_p2_ratios = [3, 2, 2, 1.5, 1.5]
fig2, axes2 = plt.subplots(
    len(_p2_ratios), 1,
    figsize=A4_PORTRAIT,
    height_ratios=_p2_ratios,
    sharex=True,
)

# ── Panel 1: Close + Bollinger Bands ──
ax = axes2[0]
ax.plot(dates, close, color='black', linewidth=CHART_STYLE['linewidth'],
        label='Close', zorder=3)

if 'BB_upper' in indicators:
    ax.plot(dates, indicators['BB_middle'], color='#1f77b4',
            linewidth=0.5, linestyle='--', label='BB Mid', alpha=0.7)
    ax.plot(dates, indicators['BB_upper'], color='gray',
            linewidth=0.5, linestyle='--', alpha=0.5)
    ax.plot(dates, indicators['BB_lower'], color='gray',
            linewidth=0.5, linestyle='--', alpha=0.5)
    ax.fill_between(dates, indicators['BB_upper'], indicators['BB_lower'],
                    alpha=0.08, color='#1f77b4', label='BB Band')

ax.set_ylabel('수익률 (%)' if is_yield else '가격',
              fontsize=CHART_STYLE['label_fontsize'])
ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'],
          ncol=2, framealpha=0.7)
ax.set_title(f'{CFG.target} — 모멘텀 지표 ({CFG.data_source})',
             fontsize=CHART_STYLE['title_fontsize'], fontweight='bold')
_style_ax(ax)

# ── Panel 2: RSI ──
ax = axes2[1]
if 'RSI' in indicators:
    rsi = indicators['RSI']
    ax.plot(dates, rsi, color='#9467bd',
            linewidth=CHART_STYLE['linewidth'], label='RSI')
    ax.axhline(INDICATOR_CONFIG.rsi_overbought, color='red',
               linewidth=0.4, linestyle='--', alpha=0.6)
    ax.axhline(INDICATOR_CONFIG.rsi_oversold, color='green',
               linewidth=0.4, linestyle='--', alpha=0.6)
    # overbought/oversold fill
    ax.fill_between(dates, INDICATOR_CONFIG.rsi_overbought, 100,
                    alpha=0.06, color='red')
    ax.fill_between(dates, 0, INDICATOR_CONFIG.rsi_oversold,
                    alpha=0.06, color='green')
    ax.set_ylim(0, 100)
    ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'])
ax.set_ylabel('RSI', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 3: Stochastic %K / %D ──
ax = axes2[2]
if 'Stoch_K' in indicators:
    ax.plot(dates, indicators['Stoch_K'], color='#1f77b4',
            linewidth=CHART_STYLE['linewidth'], label='%K')
    ax.plot(dates, indicators['Stoch_D'], color='#d62728',
            linewidth=CHART_STYLE['linewidth'], label='%D')
    ax.axhline(80, color='red', linewidth=0.4, linestyle='--', alpha=0.6)
    ax.axhline(20, color='green', linewidth=0.4, linestyle='--', alpha=0.6)
    ax.fill_between(dates, 80, 100, alpha=0.06, color='red')
    ax.fill_between(dates, 0, 20, alpha=0.06, color='green')
    ax.set_ylim(0, 100)
    ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'])
ax.set_ylabel('Stochastic', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 4: Williams %R ──
ax = axes2[3]
if 'Williams_R' in indicators:
    ax.plot(dates, indicators['Williams_R'], color='#8c564b',
            linewidth=CHART_STYLE['linewidth'])
    ax.axhline(-20, color='red', linewidth=0.4, linestyle='--', alpha=0.6)
    ax.axhline(-80, color='green', linewidth=0.4, linestyle='--', alpha=0.6)
    ax.fill_between(dates, -20, 0, alpha=0.06, color='red')
    ax.fill_between(dates, -100, -80, alpha=0.06, color='green')
    ax.set_ylim(-100, 0)
ax.set_ylabel('Williams %R', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 5: Bollinger %B + Bandwidth ──
ax = axes2[4]
if 'BB_pctb' in indicators:
    ax.plot(dates, indicators['BB_pctb'], color='#1f77b4',
            linewidth=CHART_STYLE['linewidth'], label='%B')
    ax.axhline(1.0, color='red', linewidth=0.3, linestyle=':', alpha=0.5)
    ax.axhline(0.0, color='green', linewidth=0.3, linestyle=':', alpha=0.5)
    ax.axhline(0.5, color='gray', linewidth=0.3, linestyle=':', alpha=0.4)
    ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'])

if 'BB_bandwidth' in indicators:
    ax2 = ax.twinx()
    ax2.plot(dates, indicators['BB_bandwidth'], color='#ff7f0e',
             linewidth=CHART_STYLE['linewidth'], alpha=0.7, label='Bandwidth')
    ax2.set_ylabel('Bandwidth (%)', fontsize=CHART_STYLE['label_fontsize'])
    ax2.tick_params(axis='y', labelsize=CHART_STYLE['tick_fontsize'])
    ax2.spines['top'].set_visible(False)
    ax2.legend(loc='upper right', fontsize=CHART_STYLE['legend_fontsize'])

ax.set_ylabel('Bollinger %B', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)
setup_date_axis(ax, dates)

_finish_figure(fig2, 'momentum')
plt.show()


# ══════════════════════════════════════════════════════════════
# Page 3: 변동성 + 거래량 (A4 Portrait)
# ══════════════════════════════════════════════════════════════

# 거래량 패널 존재 여부에 따라 레이아웃 결정
_vol_panels_available = []
if has_volume and 'OBV' in indicators:
    _vol_panels_available.append('OBV')
if has_volume and 'CMF' in indicators:
    _vol_panels_available.append('CMF')
if has_volume and 'AD_Line' in indicators:
    _vol_panels_available.append('AD_Line')

# 기본 3패널 (Close+Vol, ATR, Chaikin) + 거래량 지표 수만큼 추가
_p3_base = [3, 1.5, 1.5]
_p3_vol_ratios = [1.25] * len(_vol_panels_available)
_p3_ratios = _p3_base + _p3_vol_ratios
_p3_count = len(_p3_ratios)

fig3, axes3 = plt.subplots(
    _p3_count, 1,
    figsize=A4_PORTRAIT,
    height_ratios=_p3_ratios,
    sharex=True,
)
if _p3_count == 1:
    axes3 = [axes3]

# ── Panel 1: Close + Volume (dual y-axis) ──
ax = axes3[0]
ax.plot(dates, close, color='black', linewidth=CHART_STYLE['linewidth'],
        label='Close', zorder=3)

if has_volume:
    ax_vol = ax.twinx()
    vol = ohlcv_df['volume']
    ax_vol.bar(dates, vol, width=0.8, color='#7f7f7f', alpha=0.25,
               label='Volume', zorder=1)
    ax_vol.set_ylabel('거래량', fontsize=CHART_STYLE['label_fontsize'])
    ax_vol.tick_params(axis='y', labelsize=CHART_STYLE['tick_fontsize'])
    ax_vol.spines['top'].set_visible(False)
    # 거래량 축 상단 30% 영역에 표시 (가격 차트와 겹치지 않게)
    if vol.max() > 0:
        ax_vol.set_ylim(0, vol.max() * 3.5)
    ax_vol.legend(loc='upper right', fontsize=CHART_STYLE['legend_fontsize'])

ax.set_ylabel('수익률 (%)' if is_yield else '가격',
              fontsize=CHART_STYLE['label_fontsize'])
ax.legend(loc='upper left', fontsize=CHART_STYLE['legend_fontsize'])
ax.set_title(f'{CFG.target} — 변동성 + 거래량 지표 ({CFG.data_source})',
             fontsize=CHART_STYLE['title_fontsize'], fontweight='bold')
_style_ax(ax)

# ── Panel 2: ATR ──
ax = axes3[1]
if 'ATR' in indicators:
    ax.plot(dates, indicators['ATR'], color='#ff7f0e',
            linewidth=CHART_STYLE['linewidth'])
elif current_tier == '2':
    ax.text(0.5, 0.5, 'ATR: OHLC 데이터 필요',
            transform=ax.transAxes, ha='center', va='center',
            fontsize=CHART_STYLE['label_fontsize'], color='gray')
ax.set_ylabel('ATR', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── Panel 3: Chaikin Volatility ──
ax = axes3[2]
if 'Chaikin_Vol' in indicators:
    ax.plot(dates, indicators['Chaikin_Vol'], color='#8c564b',
            linewidth=CHART_STYLE['linewidth'])
    ax.axhline(0, color='black', linewidth=0.3)
elif current_tier == '2':
    ax.text(0.5, 0.5, 'Chaikin Vol: OHLC 데이터 필요',
            transform=ax.transAxes, ha='center', va='center',
            fontsize=CHART_STYLE['label_fontsize'], color='gray')
ax.set_ylabel('Chaikin Vol', fontsize=CHART_STYLE['label_fontsize'])
_style_ax(ax)

# ── 거래량 패널들 (3, 4, 5) — 거래량 있을 때만 ──
if _vol_panels_available:
    _vol_panel_idx = 3

    # Panel 4: OBV
    if 'OBV' in _vol_panels_available:
        ax = axes3[_vol_panel_idx]
        ax.plot(dates, indicators['OBV'], color='teal',
                linewidth=CHART_STYLE['linewidth'])
        ax.set_ylabel('OBV', fontsize=CHART_STYLE['label_fontsize'])
        _style_ax(ax)
        _vol_panel_idx += 1

    # Panel 5: CMF
    if 'CMF' in _vol_panels_available:
        ax = axes3[_vol_panel_idx]
        cmf = indicators['CMF']
        colors_cmf = ['#2ca02c' if v >= 0 else '#d62728' for v in cmf.values]
        ax.bar(dates, cmf, width=0.8, color=colors_cmf, alpha=0.6)
        ax.axhline(0, color='black', linewidth=0.3)
        ax.set_ylabel('CMF', fontsize=CHART_STYLE['label_fontsize'])
        _style_ax(ax)
        _vol_panel_idx += 1

    # Panel 6: A/D Line
    if 'AD_Line' in _vol_panels_available:
        ax = axes3[_vol_panel_idx]
        ax.plot(dates, indicators['AD_Line'], color='#9467bd',
                linewidth=CHART_STYLE['linewidth'])
        ax.set_ylabel('A/D Line', fontsize=CHART_STYLE['label_fontsize'])
        _style_ax(ax)
        _vol_panel_idx += 1

# 하단 축 날짜 포맷
setup_date_axis(axes3[-1], dates)

_finish_figure(fig3, 'volatility_volume')
plt.show()


# ══════════════════════════════════════════════════════════════
# Page 4: 특수차트 갤러리 (A4 Landscape) — 조건부
# ══════════════════════════════════════════════════════════════

_show_page4 = (getattr(CFG, 'show_special_charts', False)
               and has_ohlc)

if _show_page4:
    # 주석 텍스트
    _annotations = {
        'candlestick': '녹색=금리하락(채권강세), 적색=금리상승(채권약세)',
        'pnf': 'X=금리상승, O=금리하락. 시간축 무관, 추세전환만 표시',
        'three_line': '최근3봉 돌파시만 신규봉 생성. 노이즈 제거',
        'counter_clock': '우상향→매집, 좌하향→분산. 시계 역방향 순환',
        'volume_profile': 'POC(적색선)=최대거래 가격대. 지지/저항 판별',
        'candle_volume': '캔들+거래량 동시 표시. 거래량 급증 구간 주목',
    }

    fig4, axes4 = plt.subplots(
        2, 3,
        figsize=A4_LANDSCAPE,
    )

    fig4.suptitle(f'{CFG.target} — 특수차트 갤러리',
                  fontsize=CHART_STYLE['title_fontsize'], fontweight='bold',
                  y=0.98)

    # 최근 60일 슬라이스
    _last60 = ohlcv_df.iloc[-60:] if len(ohlcv_df) > 60 else ohlcv_df

    # (0,0) Candlestick
    ax = axes4[0, 0]
    try:
        chart_renderer.candlestick(_last60, ax=ax,
                                   title='캔들차트 (최근 60일)',
                                   is_yield=is_yield)
    except Exception:
        ax.text(0.5, 0.5, '캔들차트 렌더링 실패',
                transform=ax.transAxes, ha='center', va='center',
                fontsize=CHART_STYLE['label_fontsize'], color='gray')
    ax.text(0.5, -0.10, _annotations['candlestick'],
            transform=ax.transAxes, ha='center',
            fontsize=6, style='italic', color='#555555')
    _style_ax(ax)

    # (0,1) Point & Figure
    ax = axes4[0, 1]
    try:
        chart_renderer.point_and_figure(close, ax=ax,
                                        title='Point & Figure')
    except Exception:
        ax.text(0.5, 0.5, 'P&F 렌더링 실패',
                transform=ax.transAxes, ha='center', va='center',
                fontsize=CHART_STYLE['label_fontsize'], color='gray')
    ax.text(0.5, -0.10, _annotations['pnf'],
            transform=ax.transAxes, ha='center',
            fontsize=6, style='italic', color='#555555')
    _style_ax(ax)

    # (0,2) Three Line Break
    ax = axes4[0, 2]
    try:
        chart_renderer.three_line_break(close, ax=ax,
                                        title='삼선전환도')
    except Exception:
        ax.text(0.5, 0.5, '삼선전환도 렌더링 실패',
                transform=ax.transAxes, ha='center', va='center',
                fontsize=CHART_STYLE['label_fontsize'], color='gray')
    ax.text(0.5, -0.10, _annotations['three_line'],
            transform=ax.transAxes, ha='center',
            fontsize=6, style='italic', color='#555555')
    _style_ax(ax)

    # (1,0) Counter Clockwise — 거래량 필요
    ax = axes4[1, 0]
    if has_volume:
        try:
            chart_renderer.counter_clockwise(close, ohlcv_df['volume'],
                                             ax=ax, title='역시계곡선')
        except Exception:
            ax.text(0.5, 0.5, '역시계곡선 렌더링 실패',
                    transform=ax.transAxes, ha='center', va='center',
                    fontsize=CHART_STYLE['label_fontsize'], color='gray')
        ax.text(0.5, -0.10, _annotations['counter_clock'],
                transform=ax.transAxes, ha='center',
                fontsize=6, style='italic', color='#555555')
    else:
        ax.text(0.5, 0.5, '역시계곡선: 거래량 데이터 필요',
                transform=ax.transAxes, ha='center', va='center',
                fontsize=CHART_STYLE['label_fontsize'], color='gray')
        ax.set_title('역시계곡선', fontsize=CHART_STYLE['label_fontsize'])
    _style_ax(ax)

    # (1,1) Volume Profile — 거래량 필요
    ax = axes4[1, 1]
    if has_volume:
        try:
            chart_renderer.volume_profile(close, ohlcv_df['volume'],
                                          ax=ax, title='볼륨프로파일')
        except Exception:
            ax.text(0.5, 0.5, '볼륨프로파일 렌더링 실패',
                    transform=ax.transAxes, ha='center', va='center',
                    fontsize=CHART_STYLE['label_fontsize'], color='gray')
        ax.text(0.5, -0.10, _annotations['volume_profile'],
                transform=ax.transAxes, ha='center',
                fontsize=6, style='italic', color='#555555')
    else:
        ax.text(0.5, 0.5, '볼륨프로파일: 거래량 데이터 필요',
                transform=ax.transAxes, ha='center', va='center',
                fontsize=CHART_STYLE['label_fontsize'], color='gray')
        ax.set_title('볼륨프로파일', fontsize=CHART_STYLE['label_fontsize'])
    _style_ax(ax)

    # (1,2) Candle Volume — 거래량 + OHLC 필요
    ax = axes4[1, 2]
    if has_volume:
        try:
            # candle_volume expects (ax_price, ax_vol) but we only have
            # a single axis in the grid — render candlestick with volume overlay
            chart_renderer.candlestick(_last60, ax=ax,
                                       title='캔들볼륨 (최근 60일)',
                                       is_yield=is_yield)
            # overlay volume as secondary axis
            ax_cv = ax.twinx()
            vol_60 = _last60['volume']
            _cv_dates = mdates.date2num(_last60.index.to_pydatetime())
            ax_cv.bar(_cv_dates, vol_60, width=0.4, color='gray',
                      alpha=0.2, zorder=0)
            ax_cv.set_ylabel('거래량', fontsize=5)
            ax_cv.tick_params(axis='y', labelsize=5)
            if vol_60.max() > 0:
                ax_cv.set_ylim(0, vol_60.max() * 4)
            ax_cv.spines['top'].set_visible(False)
        except Exception:
            ax.text(0.5, 0.5, '캔들볼륨 렌더링 실패',
                    transform=ax.transAxes, ha='center', va='center',
                    fontsize=CHART_STYLE['label_fontsize'], color='gray')
        ax.text(0.5, -0.10, _annotations['candle_volume'],
                transform=ax.transAxes, ha='center',
                fontsize=6, style='italic', color='#555555')
    else:
        ax.text(0.5, 0.5, '캔들볼륨: 거래량 데이터 필요',
                transform=ax.transAxes, ha='center', va='center',
                fontsize=CHART_STYLE['label_fontsize'], color='gray')
        ax.set_title('캔들볼륨', fontsize=CHART_STYLE['label_fontsize'])
    _style_ax(ax)

    _finish_figure(fig4, 'special_charts', tight_pad=2.0)
    plt.show()


# ══════════════════════════════════════════════════════════════
# Summary
# ══════════════════════════════════════════════════════════════

print(f"\n{'='*60}")
print(f"종목: {CFG.target} | 소스: {CFG.data_source}")
print(f"기간: {ohlcv_df.index[0].date()} ~ {ohlcv_df.index[-1].date()}")
print(f"산출 지표 수: {len(indicators)}")
print(f"생성 차트: {len(_all_figs)}페이지")
print(f"{'='*60}")


In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 5: 시그널 종합 대시보드
# ══════════════════════════════════════════════════════════════

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy.stats import percentileofscore

# ── 게이지 기간 업데이트 ─────────────────────────────────────
INDICATOR_CONFIG.gauge_short = CFG.gauge_short
INDICATOR_CONFIG.gauge_medium = CFG.gauge_medium
INDICATOR_CONFIG.gauge_long = CFG.gauge_long

# ── 시그널 분류 ──────────────────────────────────────────────
signals = signal_classifier.classify_all(indicators, ohlcv_df['close'])
composite = signal_classifier.composite_score(signals)

# ── 백분위 재계산 (방어적: 'full' 키 보장 + 정확도 향상) ──────
SIGNAL_INDICATOR_MAP = {
    'RSI': 'RSI', 'Stochastic': 'Stoch_K', 'Williams %R': 'Williams_R',
    'MACD': 'MACD_hist', 'DMI': 'DMI_adx', 'CCI': 'CCI',
    'Bollinger': 'BB_pctb', 'ATR': 'ATR', 'OBV': 'OBV', 'CMF': 'CMF',
    'SONAR': 'SONAR', 'Chaikin Vol': 'Chaikin_Vol', 'A/D Line': 'AD_Line',
}

def _recompute_gauges(series, short_w, med_w, long_w):
    """indicators dict에서 직접 백분위 재계산"""
    val = series.iloc[-1]
    if np.isnan(val):
        return {'full': 0.5, 'short': 0.5, 'medium': 0.5, 'long': 0.5}

    def _pct(subset):
        s = subset.dropna()
        if len(s) < 5:
            return 0.5
        return percentileofscore(s.values, val, kind='rank') / 100.0

    return {
        'full': _pct(series),
        'short': _pct(series.iloc[-short_w:]),
        'medium': _pct(series.iloc[-med_w:]),
        'long': _pct(series.iloc[-long_w:]),
    }

for _sig in signals:
    _ind_key = SIGNAL_INDICATOR_MAP.get(_sig.name)
    if _ind_key and _ind_key in indicators and _sig.signal != 'n/a':
        _series = indicators[_ind_key]
        if len(_series.dropna()) >= 5:
            _sig.rel_gauges = _recompute_gauges(
                _series, CFG.gauge_short, CFG.gauge_medium, CFG.gauge_long
            )

# ══════════════════════════════════════════════════════════════
# Goldman Sachs 스타일 색상
# ══════════════════════════════════════════════════════════════
C_BUY     = '#0a7c42'   # 포레스트 그린
C_SELL    = '#b82025'   # 딥 레드
C_NEUTRAL = '#555555'   # 미디엄 그레이
C_NA      = '#aaaaaa'
C_HDR_BG  = '#2d2d2d'   # 차콜
C_HDR_FG  = '#f5f5f5'   # 오프화이트
C_CAT_BG  = '#f0f0f0'   # 라이트 그레이
C_CAT_FG  = '#333333'
C_ROW_ODD = '#ffffff'
C_ROW_EVEN= '#f9f9f9'
C_BORDER  = '#d0d0d0'   # 라이트 보더
C_BG      = '#ffffff'
C_TITLE   = '#1a1a1a'   # 니어 블랙
C_ACCENT  = '#0054a6'   # 블루 악센트

# ── 카테고리 정의 ──
CATEGORY_MAP = {
    'trend':      ('추세',    ['MACD', 'DMI', 'CCI', 'SONAR', 'Bollinger']),
    'momentum':   ('모멘텀',  ['RSI', 'Stochastic', 'Williams %R']),
    'volatility': ('변동성',  ['ATR', 'Chaikin Vol', 'Bollinger']),
    'volume':     ('거래량',  ['OBV', 'CMF', 'A/D Line']),
}
CAT_ORDER = ['trend', 'momentum', 'volatility', 'volume']

# ── 기간 라벨 ──
n_data = len(ohlcv_df)
GAUGE_KEYS = ['full', 'short', 'medium', 'long']
GAUGE_LABELS = [
    f'지정기간({n_data}d)',
    f'단기({CFG.gauge_short}d)',
    f'중기({CFG.gauge_medium}d)',
    f'장기({CFG.gauge_long}d)',
]

# ── 방향 반전 지표 (값 상승 = 매도 방향인 지표) ──
INVERTED_INDICATORS = {'MACD', 'OBV', 'CMF', 'SONAR', 'A/D Line'}


def _sig_color(s):
    if s == 'buy':     return C_BUY
    if s == 'sell':    return C_SELL
    if s == 'neutral': return C_NEUTRAL
    return C_NA

def _sig_kr(s):
    return {'buy': '매수', 'sell': '매도', 'neutral': '중립', 'n/a': '-'}.get(s, s)

def _pct_str(v):
    if v is None or np.isnan(v):
        return '-'
    return f'{v*100:.0f}%'

def _pct_color(v):
    """백분위 → 색상 (0%=매수/녹, 50%=중립/회, 100%=매도/적)"""
    if v is None or np.isnan(v):
        return C_NA
    if v <= 0.30:   return C_BUY
    elif v >= 0.70: return C_SELL
    return C_NEUTRAL

def _direction_display(sig):
    """시그널 맥락 기반 방향 화살표 + 색상
    ▲ = 매수 유리 방향 (녹), ▼ = 매도 유리 방향 (적)
    """
    if sig.direction == 'stable' or sig.signal in ('neutral', 'n/a'):
        return '-', C_NEUTRAL
    value_rising = (sig.direction == 'strengthening')
    is_inverted = sig.name in INVERTED_INDICATORS
    buy_favorable = value_rising if not is_inverted else not value_rising
    arrow = '\u25b2' if buy_favorable else '\u25bc'
    color = C_BUY if buy_favorable else C_SELL
    return arrow, color


# ══════════════════════════════════════════════════════════════
# 시그널 그룹핑
# ══════════════════════════════════════════════════════════════
signal_by_name = {s.name: s for s in signals}
grouped = []
_used = set()
for ck in CAT_ORDER:
    clbl, pref = CATEGORY_MAP[ck]
    csigs = []
    for pn in pref:
        if pn in signal_by_name and pn not in _used:
            csigs.append(signal_by_name[pn])
            _used.add(pn)
    for s in signals:
        if s.category == ck and s.name not in _used:
            csigs.append(s)
            _used.add(s.name)
    if csigs:
        grouped.append((clbl, csigs))
remaining = [s for s in signals if s.name not in _used]
if remaining:
    grouped.append(('기타', remaining))

total_indicators = sum(len(sigs) for _, sigs in grouped)
total_rows = total_indicators + len(grouped)

# ══════════════════════════════════════════════════════════════
# Figure — 테이블 스타일
# ══════════════════════════════════════════════════════════════

N_COLS = 8
COL_WIDTHS = [0.14, 0.12, 0.08, 0.05, 0.1525, 0.1525, 0.1525, 0.1525]
COL_LABELS = ['지표', '현재값', '시그널', '방향'] + GAUGE_LABELS

ROW_H = 0.035
HDR_H = 0.036
CAT_H = 0.028
TITLE_H = 0.055
SCORE_H = 0.065
FOOTER_H = 0.025

body_h = total_rows * ROW_H + len(grouped) * CAT_H
total_h = TITLE_H + HDR_H + body_h + SCORE_H + FOOTER_H + 0.02

fig_w, fig_h_max = A4_PORTRAIT
fig_h = min(fig_h_max, max(6.0, total_h * fig_h_max))

fig = plt.figure(figsize=(fig_w, fig_h), facecolor=C_BG)
ax = fig.add_axes([0.03, 0.02, 0.94, 0.96])
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')


def _draw_rect(x, y, w, h, fc='none', ec=C_BORDER, lw=0.5):
    ax.add_patch(mpatches.FancyBboxPatch(
        (x, y), w, h, boxstyle='square,pad=0',
        facecolor=fc, edgecolor=ec, linewidth=lw))

def _draw_text(x, y, txt, **kw):
    defaults = dict(fontsize=6.5, va='center', ha='center',
                    transform=ax.transData, clip_on=False)
    defaults.update(kw)
    ax.text(x, y, txt, **defaults)

def _draw_pct_bar(x, y, w, h, pct):
    """미니 수평 게이지 (셀 내부)"""
    if pct is None or np.isnan(pct):
        return
    pct = np.clip(pct, 0, 1)
    bar_w = w * 0.82
    bar_h = h * 0.28
    bar_x = x + (w - bar_w) / 2
    bar_y = y + h * 0.12

    # 배경 바
    ax.add_patch(mpatches.Rectangle(
        (bar_x, bar_y), bar_w, bar_h,
        facecolor='#e8e8e8', edgecolor='#cccccc', linewidth=0.3))
    # 채움 바
    if pct > 0.005:
        fill_color = _pct_color(pct)
        ax.add_patch(mpatches.Rectangle(
            (bar_x, bar_y), bar_w * pct, bar_h,
            facecolor=fill_color, edgecolor='none', alpha=0.35))
    # 위치 마커 (세로선)
    marker_x = bar_x + bar_w * pct
    ax.plot([marker_x, marker_x],
            [bar_y - h * 0.02, bar_y + bar_h + h * 0.02],
            color=_pct_color(pct), linewidth=1.2, solid_capstyle='round')
    # 수치 텍스트 (바 위)
    _draw_text(x + w / 2, y + h * 0.72, _pct_str(pct),
               fontsize=5.8, fontweight='bold', color=_pct_color(pct))


# ── 제목 영역 ──
date_str = ohlcv_df.index[-1].strftime('%Y-%m-%d')
data_label = '수익률' if is_yield else '가격'
score_val = composite['score']

title_y = 1.0 - TITLE_H
_draw_text(0.5, title_y + TITLE_H * 0.65,
           f'{CFG.target}  기술적 시그널 종합',
           fontsize=11, fontweight='bold', ha='center', color=C_TITLE)
_draw_text(0.5, title_y + TITLE_H * 0.22,
           f'{date_str}   |   {data_label}   |   Tier {current_tier}   |   {CFG.data_source}',
           fontsize=6.5, ha='center', color='#888888')

# ── 헤더 행 (차콜 배경) ──
hdr_y = title_y - HDR_H
x_pos = 0.0
for ci in range(N_COLS):
    w = COL_WIDTHS[ci]
    _draw_rect(x_pos, hdr_y, w, HDR_H, fc=C_HDR_BG, ec=C_HDR_BG)
    _draw_text(x_pos + w/2, hdr_y + HDR_H/2, COL_LABELS[ci],
               fontsize=5.8, fontweight='bold', color=C_HDR_FG)
    x_pos += w

# ── 데이터 행 ──
cur_y = hdr_y
row_idx = 0

for cat_label, cat_sigs in grouped:
    # 카테고리 행
    cur_y -= CAT_H
    _draw_rect(0, cur_y, 1.0, CAT_H, fc=C_CAT_BG, ec=C_BORDER, lw=0.3)
    # 좌측 악센트 바
    ax.add_patch(mpatches.Rectangle(
        (0, cur_y), 0.004, CAT_H, facecolor=C_ACCENT, edgecolor='none'))
    _draw_text(0.01 + COL_WIDTHS[0]/2, cur_y + CAT_H/2,
               cat_label, fontsize=6.5, fontweight='bold',
               color=C_CAT_FG, ha='center')

    for sig in cat_sigs:
        cur_y -= ROW_H
        bg = C_ROW_ODD if row_idx % 2 == 0 else C_ROW_EVEN
        _draw_rect(0, cur_y, 1.0, ROW_H, fc=bg, ec=C_BORDER, lw=0.15)

        x = 0.0

        # Col 0: 지표명
        w = COL_WIDTHS[0]
        name_str = sig.name
        if sig.divergence:
            name_str += ' [D]'
        _draw_text(x + 0.008, cur_y + ROW_H/2, name_str,
                   fontsize=6.5, fontweight='bold', ha='left', color='#1a1a1a')
        x += w

        # Col 1: 현재값 / 설명
        w = COL_WIDTHS[1]
        if sig.signal != 'n/a':
            val_str = f'{sig.value:.2f}' if not np.isnan(sig.value) else '-'
            _draw_text(x + w/2, cur_y + ROW_H*0.62, val_str,
                       fontsize=5.8, ha='center', color='#333333')
            desc = sig.description[:16] + '..' if len(sig.description) > 18 else sig.description
            _draw_text(x + w/2, cur_y + ROW_H*0.28, desc,
                       fontsize=4.2, ha='center', color='#999999')
        else:
            _draw_text(x + w/2, cur_y + ROW_H/2, '-', fontsize=6, color=C_NA)
        x += w

        # Col 2: 시그널
        w = COL_WIDTHS[2]
        sc = _sig_color(sig.signal)
        sig_txt = _sig_kr(sig.signal)
        _draw_text(x + w/2, cur_y + ROW_H/2, sig_txt,
                   fontsize=6, fontweight='bold', color=sc)
        x += w

        # Col 3: 방향 (시그널 맥락 기반)
        w = COL_WIDTHS[3]
        arrow, arrow_c = _direction_display(sig)
        _draw_text(x + w/2, cur_y + ROW_H/2, arrow,
                   fontsize=7, fontweight='bold', color=arrow_c)
        x += w

        # Col 4~7: 백분위 게이지
        is_na = (sig.signal == 'n/a')
        for gi, gk in enumerate(GAUGE_KEYS):
            w = COL_WIDTHS[4 + gi]
            gval = sig.rel_gauges.get(gk, 0.5)
            if is_na:
                _draw_text(x + w/2, cur_y + ROW_H/2, '-',
                           fontsize=6, color=C_NA)
            else:
                _draw_pct_bar(x, cur_y, w, ROW_H, gval)
            x += w

        row_idx += 1

# ── 종합 스코어 영역 ──
cur_y -= 0.006
score_y = cur_y - SCORE_H

_draw_rect(0, score_y, 1.0, SCORE_H, fc='#f5f5f5', ec=C_HDR_BG, lw=0.6)

score_str = f'{score_val:+.2f}'
if score_val > 0.2:     sc_color = C_BUY
elif score_val < -0.2:  sc_color = C_SELL
else:                    sc_color = C_NEUTRAL

_draw_text(0.02, score_y + SCORE_H*0.58, '종합 스코어',
           fontsize=8, fontweight='bold', ha='left', color=C_TITLE)
_draw_text(0.22, score_y + SCORE_H*0.58, score_str,
           fontsize=13, fontweight='bold', ha='left', color=sc_color)

count_parts = [
    (f'매수 {composite["buy"]}', C_BUY),
    (f'매도 {composite["sell"]}', C_SELL),
    (f'중립 {composite["neutral"]}', C_NEUTRAL),
]
if composite.get('na', 0) > 0:
    count_parts.append((f'N/A {composite["na"]}', C_NA))

cx = 0.42
for ctxt, cc in count_parts:
    _draw_text(cx, score_y + SCORE_H*0.58, ctxt,
               fontsize=6.5, fontweight='bold', ha='left', color=cc)
    cx += 0.12

_draw_text(0.02, score_y + SCORE_H*0.20,
           f'유효: {composite.get("valid",0)}/{composite.get("total",len(signals))}   |   '
           f'{ohlcv_df.index[0].strftime("%Y-%m-%d")} ~ {date_str} ({n_data}d)',
           fontsize=5.5, ha='left', color='#999999')

# ── 범례 ──
footer_y = score_y - FOOTER_H - 0.003
_draw_text(0.5, footer_y + FOOTER_H*0.5,
           '0%=하위(채권매수) \u2192 100%=상위(채권매도)   |   '
           '\u25b2 매수유리  \u25bc 매도유리   |   [D] 다이버전스',
           fontsize=4.8, ha='center', color='#aaaaaa', style='italic')

# ── 저장 & 표시 ──
_all_figs.append((fig, 'signal_dashboard'))
plt.show()

# ── 텍스트 요약 ──
print(f"\n{'='*60}")
print(f"시그널 대시보드: {CFG.target}")
print(f"종합 스코어: {composite['score']:+.2f}")
print(f"매수: {composite['buy']} / 매도: {composite['sell']} / 중립: {composite['neutral']}")
print(f"{'='*60}")

# ── 백분위 상세 출력 (검증용) ──
print(f"\n{'─'*60}")
print(f"백분위 상세 (0%=하위/매수 → 100%=상위/매도):")
print(f"  {'지표':15s}  {'지정기간':>7s}  {'단기':>7s}  {'중기':>7s}  {'장기':>7s}  시그널")
print(f"  {'─'*15}  {'─'*7}  {'─'*7}  {'─'*7}  {'─'*7}  {'─'*6}")
for _s in signals:
    _g = _s.rel_gauges
    _f = lambda k: f"{_g.get(k, -1)*100:5.1f}%" if _g.get(k, -1) >= 0 else "  N/A "
    print(f"  {_s.name:15s}  {_f('full')}  {_f('short')}  {_f('medium')}  {_f('long')}  {_s.signal}")
print(f"{'─'*60}")


In [ ]:
# ══════════════════════════════════════════════════════════════
# Cell 6: Excel/이미지 저장
# ══════════════════════════════════════════════════════════════

import pandas as pd
from datetime import datetime
from pathlib import Path

today = datetime.now().strftime("%Y%m%d")

# ── 저장 디렉토리 확인/생성 ──────────────────────────────────
try:
    CFG.save_dir.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"[WARNING] 저장 디렉토리 생성 실패: {e}")

saved_files = []

# ══════════════════════════════════════════════════════════════
# Excel 저장
# ══════════════════════════════════════════════════════════════
if CFG.save_excel:
    excel_path = CFG.save_dir / f"{CFG.target}_{CFG.data_source}_{today}.xlsx"
    try:
        with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:

            # ── Sheet 1: 지표원시값 ──────────────────────────
            if indicators:
                ind_df = pd.DataFrame(indicators, index=ohlcv_df.index)
                ind_df.index.name = "Date"
                ind_df.to_excel(writer, sheet_name="지표원시값")
            else:
                pd.DataFrame({"info": ["지표 데이터 없음"]}).to_excel(
                    writer, sheet_name="지표원시값", index=False
                )

            # ── Sheet 2: 시그널요약 ──────────────────────────
            if signals:
                sig_records = []
                for s in signals:
                    rg = getattr(s, "rel_gauges", {})
                    rec = {
                        "name": s.name,
                        "category": s.category,
                        "value": s.value,
                        "signal": s.signal,
                        "description": s.description,
                        "abs_gauge": s.abs_gauge,
                        "rel_gauge_short": rg.get("short", None),
                        "rel_gauge_medium": rg.get("medium", None),
                        "rel_gauge_long": rg.get("long", None),
                        "direction": s.direction,
                        "divergence": s.divergence,
                    }
                    sig_records.append(rec)
                sig_df = pd.DataFrame(sig_records)
                sig_df.to_excel(writer, sheet_name="시그널요약", index=False)
            else:
                pd.DataFrame({"info": ["시그널 데이터 없음"]}).to_excel(
                    writer, sheet_name="시그널요약", index=False
                )

            # ── Sheet 3: 종합스코어 ──────────────────────────
            comp_row = {
                "score": composite.get("score", None),
                "buy": composite.get("buy", 0),
                "sell": composite.get("sell", 0),
                "neutral": composite.get("neutral", 0),
                "na": composite.get("na", 0),
                "total": composite.get("total", 0),
                "valid": composite.get("valid", 0),
                "target": CFG.target,
                "data_source": CFG.data_source,
                "date": today,
                "tier": current_tier,
            }
            comp_df = pd.DataFrame([comp_row])
            comp_df.to_excel(writer, sheet_name="종합스코어", index=False)

        saved_files.append(("Excel", excel_path))
        print(f"[OK] Excel 저장 완료: {excel_path.name}")

    except Exception as e:
        print(f"[ERROR] Excel 저장 실패: {e}")

# ══════════════════════════════════════════════════════════════
# 이미지 저장 (PNG + PDF)
# ══════════════════════════════════════════════════════════════
if CFG.save_graphs:
    png_count = 0

    if _all_figs:
        # ── 개별 PNG 저장 ────────────────────────────────────
        for fig, name in _all_figs:
            try:
                png_path = CFG.save_dir / f"{CFG.target}_{name}_{today}.png"
                fig.savefig(png_path, dpi=150, bbox_inches="tight", facecolor="white")
                png_count += 1
            except Exception as e:
                print(f"[WARNING] PNG 저장 실패 ({name}): {e}")

        if png_count:
            saved_files.append(("PNG", f"{png_count}개"))
            print(f"[OK] PNG 저장 완료: {png_count}개")

        # ── 통합 PDF 저장 ────────────────────────────────────
        try:
            from matplotlib.backends.backend_pdf import PdfPages

            pdf_path = CFG.save_dir / f"{CFG.target}_{CFG.data_source}_{today}_전체.pdf"
            with PdfPages(pdf_path) as pdf:
                for fig, name in _all_figs:
                    pdf.savefig(fig, bbox_inches="tight", facecolor="white")
            saved_files.append(("PDF", pdf_path))
            print(f"[OK] PDF 저장 완료: {pdf_path.name}")
        except Exception as e:
            print(f"[WARNING] PDF 저장 실패: {e}")
    else:
        print("[INFO] 저장할 그래프가 없습니다 (_all_figs 비어 있음)")

# ══════════════════════════════════════════════════════════════
# 저장 요약
# ══════════════════════════════════════════════════════════════
print(f"\n{'='*60}")
print(f"저장 완료")
print(f"{'='*60}")
if CFG.save_excel:
    print(f"  Excel: {excel_path}")
if CFG.save_graphs:
    print(f"  이미지: {len(_all_figs)}개 PNG + 1 PDF")
    print(f"  디렉토리: {CFG.save_dir.resolve()}")
print(f"{'='*60}")
